In [762]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy.stats.stats import pearsonr

df = pd.read_csv('newTrain.csv')
dfold = pd.read_csv('newTrain.csv')
df["PurchDate"] = df["PurchDate"].astype("datetime64")
dfold["PurchDate"] = dfold["PurchDate"].astype("datetime64")

In [763]:
df.head()

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Model,Wheeldrive,Doors,SubModel
0,326,1,2010-10-25,ADESA,2008,2,KIA,NaN,EX,NaN,BLUE,AUTO,2.0,Covers,61184,OTHER ASIAN,MEDIUM,OTHER,6796.0,7555.0,10140.0,10805.0,6628.0,7535.0,9605.0,10650.0,NaN,NaN,5546,33619,FL,6100.0,0,533,SPECTRA,NaN,4D,SEDAN
1,42991,0,2009-05-27,MANHEIM,2006,3,DODGE,2.7L,SXT,FFV,SILVER,AUTO,2.0,Covers,92383,AMERICAN,MEDIUM,CHRYSLER,5480.0,6461.0,6418.0,7478.0,5480.0,6461.0,6418.0,7478.0,NaN,NaN,99750,32124,FL,4000.0,0,1630,STRATUS,NaN,4D,SEDAN
2,55273,0,2010-03-18,OTHER,2008,2,DODGE,NaN,SE,NaN,BLACK,AUTO,2.0,Covers,45965,AMERICAN,MEDIUM,CHRYSLER,8067.0,8580.0,11073.0,11652.0,7497.0,8102.0,10721.0,11350.0,NaN,NaN,99761,74135,OK,7500.0,0,693,CALIBER,NaN,4D,WAGON
3,29058,0,2009-08-06,OTHER,2005,4,FORD,3.9L,S,NaN,BROWN,AUTO,2.0,Covers,80169,AMERICAN,VAN,FORD,4817.0,6658.0,5702.0,7691.0,4817.0,6658.0,5702.0,7691.0,NaN,NaN,99761,85018,AZ,4725.0,0,1633,FREESTAR,2WD,NaN,PASSENGER
4,34991,0,2009-04-03,MANHEIM,2005,4,CHRYSLER,V6,BAS,NaN,BLUE,AUTO,1.0,Alloy,77372,AMERICAN,VAN,CHRYSLER,5430.0,6633.0,6364.0,7664.0,5370.0,6811.0,6300.0,7856.0,NaN,NaN,20833,75236,TX,5670.0,0,1623,TOWN & COUNTRY,2WD,NaN,MINIVAN


## Trasformazine dei valori incompleti per gli MMR (1 e 0) in null

In [764]:
Variable='MMRAcquisitionAuctionAveragePrice'
df[Variable].replace(0, np.nan, inplace=True)
df[Variable].replace(1, np.nan, inplace=True)

In [765]:
Variable='MMRAcquisitionAuctionCleanPrice'
df[Variable].replace(0, np.nan, inplace=True)
df[Variable].replace(1, np.nan, inplace=True)

In [766]:
Variable='MMRAcquisitionRetailAveragePrice'
df[Variable].replace(0, np.nan, inplace=True)
df[Variable].replace(1, np.nan, inplace=True)

In [767]:
Variable='MMRAcquisitonRetailCleanPrice'
df[Variable].replace(0, np.nan, inplace=True)
df[Variable].replace(1, np.nan, inplace=True)

In [768]:
Variable='MMRCurrentAuctionAveragePrice'
df[Variable].replace(0, np.nan, inplace=True)
df[Variable].replace(1, np.nan, inplace=True)

In [769]:
Variable='MMRCurrentAuctionCleanPrice'
df[Variable].replace(0, np.nan, inplace=True)
df[Variable].replace(1, np.nan, inplace=True)

In [770]:
Variable='MMRCurrentRetailAveragePrice'
df[Variable].replace(0, np.nan, inplace=True)
df[Variable].replace(1, np.nan, inplace=True)

In [771]:
Variable='MMRCurrentRetailCleanPrice'
df[Variable].replace(0, np.nan, inplace=True)
df[Variable].replace(1, np.nan, inplace=True)

In [772]:
dig1 = set(df[df['MMRAcquisitionAuctionAveragePrice'].isnull()]['RefId'])
dig2 = set(df[df['MMRAcquisitionAuctionCleanPrice'].isnull()]['RefId'])
dig3 = set(df[df['MMRAcquisitionRetailAveragePrice'].isnull()]['RefId'])
dig4 = set(df[df['MMRAcquisitonRetailCleanPrice'].isnull()]['RefId'])
dig5 = set(df[df['MMRCurrentAuctionAveragePrice'].isnull()]['RefId'])
dig6 = set(df[df['MMRCurrentAuctionCleanPrice'].isnull()]['RefId'])
dig7 = set(df[df['MMRCurrentRetailAveragePrice'].isnull()]['RefId'])
dig8 = set(df[df['MMRCurrentRetailCleanPrice'].isnull()]['RefId'])

In [773]:
len(dig1.intersection(dig2,dig3,dig4))

661

In [774]:
len(dig5.intersection(dig6,dig7,dig8))

638

In [775]:
len(dig1.intersection(dig5))

371

# Modifiche Preliminari per preservare la correttezza semantica

In [776]:
df['Make'].replace(['TOYOTA SCION'], ['SCION'], inplace=True)
ValueCount=df['Make'].value_counts()
ValueCount[df['Make'].value_counts()<15]

PLYMOUTH    1
HUMMER      1
Name: Make, dtype: int64

In [777]:
df['WheelTypeID'].replace(0, np.nan, inplace=True)

In [778]:
df['Nationality'].replace(['OTHER', 'OTHER ASIAN', 'TOP LINE ASIAN'], ['NOT AMERICAN', 'NOT AMERICAN', 'NOT AMERICAN'], inplace=True)
df['Nationality'].value_counts()

AMERICAN        48800
NOT AMERICAN     9582
Name: Nationality, dtype: int64

In [779]:
Variable="SubModel"
#Sostituzione dei valori nulli del submodel utilizzando la moda raggruppata per modello e anno
df[Variable] = df['SubModel'].groupby([df['Model'], df['VehYear']]).fillna(df[Variable].mode()[0])
df[Variable].isnull().sum()

0

In [780]:
Variable='Trim'
#Sostituzione dei valori nulli del trim utilizzando la moda raggruppata per modello sottomodello e anno
df[Variable] = df[Variable].groupby([df['Model'], df['SubModel'], df['VehYear']]).fillna(df[Variable].mode()[0])
df[Variable].isnull().sum()

0

In [781]:
Variable='Transmission'
#Sostituzione dei valori nulli della trasmissione utilizzando la moda raggruppata per modello sottomodello e anno
df[Variable] = df[Variable].groupby([df['Model'], df['SubModel'], df['VehYear']]).fillna(df[Variable].mode()[0])
df[Variable].isnull().sum()

0

In [782]:
Variable='WheelTypeID'
#Sostituzione dei valori nulli del wheeltypeID utilizzando la moda raggruppata per modello sottomodello e Trim
df[Variable] = df[Variable].groupby([df['Model'], df['SubModel'], df['Trim']]).fillna(df[Variable].mode()[0])
df[Variable].isnull().sum()

0

In [783]:
Variable='Nationality'
#Sostituzione dei valori nulli di Nationality utilizzando la moda raggruppata per modello sottomodello e Trim
df[Variable] = df[Variable].groupby([df['Make']]).fillna(df[Variable].mode()[0])
df[Variable].isnull().sum()

0

In [784]:
Variable='Size'
#Sostituzione dei valori nulli di Size utilizzando la moda raggruppata per modello sottomodello e Trim
df[Variable] = df[Variable].groupby([df['Model'], df['SubModel'], df['VehYear']]).fillna(df[Variable].mode()[0])
df[Variable].isnull().sum()

0

# Prezzo all'asta

In [785]:
Variable='MMRAcquisitionAuctionAveragePrice'

## dff è il dataset raggruppato con la definizione corretta di MMR

In [834]:
dff = df[Variable].groupby([df['PurchDate'], df['Model'], df['SubModel'], df['VehYear']]).var().reset_index()

## dffsenzadata è il dataset raggruppato senza tener conto della data di stima del MMR

In [835]:
dffsenzadata = df[Variable].groupby([df['Model'], df['SubModel'], df['VehYear']]).var().reset_index()

In [836]:
dff.rename(columns={Variable:'Varianza'}, inplace=True)
dff

,PurchDate,Model,SubModel,VehYear,Varianza
0,2009-01-04,1500 RAM,QUAD CAB,2002,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,NaN
2,2009-01-04,1500 RAM,QUAD CAB,2004,6949351.0
3,2009-01-04,1500 RAM,QUAD CAB,2005,NaN
4,2009-01-04,300,SEDAN,2006,0.0
...,...,...,...,...,...
40914,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,NaN
40915,2010-12-30,MALIBU,SEDAN,2005,NaN
40916,2010-12-30,SPECTRA,SEDAN,2006,NaN
40917,2010-12-30,TAURUS,SEDAN,2006,NaN


In [837]:
dff['Varianza'].isnull().sum()

31680

Una varianza pari a null indica che il raggruppamento corrisponde a una sola macchina o sono presenti solo macchine con valori null per l'MMR

In [838]:
dff['Varianza'] = dff['Varianza'].fillna(0)

# Varianza
all'incirca il 90% del database ha una varianza inferiore alla soglia scelta (250000) mentre il 71 % ha una varianza pari a 0 

In [839]:
dff.loc[dff["Varianza"]<1]["Varianza"]

0        0.0
1        0.0
3        0.0
4        0.0
5        0.0
        ... 
40914    0.0
40915    0.0
40916    0.0
40917    0.0
40918    0.0
Name: Varianza, Length: 35257, dtype: float64

In [840]:
dff.loc[dff["Varianza"]<=250000]["Varianza"]

0        0.0
1        0.0
3        0.0
4        0.0
5        0.0
        ... 
40914    0.0
40915    0.0
40916    0.0
40917    0.0
40918    0.0
Name: Varianza, Length: 38796, dtype: float64

In [841]:
dff.loc[dff["Varianza"]>250000]["Varianza"]

2        6949351.00
16        294818.25
32       1436512.50
36        322752.00
116       788768.00
            ...    
40718    1486088.00
40720     937080.50
40763    1729800.00
40784     442537.00
40814     252760.50
Name: Varianza, Length: 2123, dtype: float64

In [721]:
dff[:350]

,PurchDate,Model,SubModel,VehYear,Varianza
0,2009-01-04,1500 RAM,QUAD CAB,2002,0.0
1,2009-01-04,1500 RAM,QUAD CAB,2003,0.0
2,2009-01-04,1500 RAM,QUAD CAB,2004,6949351.0
3,2009-01-04,1500 RAM,QUAD CAB,2005,0.0
4,2009-01-04,300,SEDAN,2006,0.0
...,...,...,...,...,...
345,2009-01-09,ION,SEDAN,2005,0.0
346,2009-01-09,ION,SEDAN,2006,0.0
347,2009-01-09,LIBERTY,SUV,2003,0.0
348,2009-01-09,LIBERTY,SUV,2006,0.0


## Esempio di macchina con varianza alta
le 3 macchin dovrebbero avere MMR molto simile ma una di queste ha MMR molto ridotto nonostante il prezzo di acquisto sia lo molto simile per tutte le macchine

In [722]:
dftest = df.loc[df['Model'] == '1500 RAM']

In [723]:
dftest = dftest.loc[dftest['SubModel'] == 'QUAD CAB']

In [724]:
dftest = dftest.loc[dftest['VehYear'] == 2004]

In [725]:
dftest = dftest.loc[dftest['PurchDate'] == '2009-01-04']

In [726]:
dftest[Variable]

7963     9620.0
22899    8496.0
57704    4597.0
Name: MMRAcquisitionAuctionAveragePrice, dtype: float64

In [727]:
pd.options.display.max_columns = 40

In [728]:
dftest

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Model,Wheeldrive,Doors,SubModel
7963,46450,0,2009-01-04,MANHEIM,2004,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,81296,AMERICAN,LARGE TRUCK,CHRYSLER,9620.0,14210.0,10890.0,15847.0,9620.0,14210.0,10890.0,15847.0,NaN,NaN,19638,33809,FL,9460.0,0,1893,1500 RAM,4WD,NaN,QUAD CAB
22899,14101,0,2009-01-04,MANHEIM,2004,5,DODGE,5.7L,ST,NaN,BLACK,AUTO,3.0,Special,75251,AMERICAN,LARGE TRUCK,CHRYSLER,8496.0,10471.0,9676.0,11809.0,8496.0,10471.0,9676.0,11809.0,NaN,NaN,21053,92807,CA,9275.0,0,920,1500 RAM,2WD,NaN,QUAD CAB
57704,20577,0,2009-01-04,MANHEIM,2004,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,76276,AMERICAN,LARGE TRUCK,CHRYSLER,4597.0,6357.0,5465.0,7366.0,4597.0,6357.0,5465.0,7366.0,NaN,NaN,20392,75236,TX,9450.0,0,920,1500 RAM,2WD,NaN,QUAD CAB


fine test

In [842]:
mergedDF = pd.merge(dff, df, on=['PurchDate','Model', 'SubModel', 'VehYear'],  how='left')

In [844]:
mergedDF

,PurchDate,Model,SubModel,VehYear,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,2009-01-04,1500 RAM,QUAD CAB,2002,0.0,62472,0,ADESA,7,DODGE,5.9L,ST,NaN,BLUE,AUTO,1.0,Alloy,62209,AMERICAN,LARGE TRUCK,CHRYSLER,5872.0,7407.0,6842.0,8500.0,5872.0,7407.0,6842.0,8500.0,NaN,NaN,20234,85226,AZ,7940.0,0,1125,2WD,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,0.0,16342,0,MANHEIM,6,DODGE,5.7L,LAR,NaN,SILVER,AUTO,1.0,Alloy,79925,AMERICAN,LARGE TRUCK,CHRYSLER,7648.0,9717.0,8760.0,10994.0,7648.0,9717.0,8760.0,10994.0,NaN,NaN,19619,32824,FL,9400.0,0,1220,2WD,NaN
2,2009-01-04,1500 RAM,QUAD CAB,2004,6949351.0,46450,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,81296,AMERICAN,LARGE TRUCK,CHRYSLER,9620.0,14210.0,10890.0,15847.0,9620.0,14210.0,10890.0,15847.0,NaN,NaN,19638,33809,FL,9460.0,0,1893,4WD,NaN
3,2009-01-04,1500 RAM,QUAD CAB,2004,6949351.0,14101,0,MANHEIM,5,DODGE,5.7L,ST,NaN,BLACK,AUTO,3.0,Special,75251,AMERICAN,LARGE TRUCK,CHRYSLER,8496.0,10471.0,9676.0,11809.0,8496.0,10471.0,9676.0,11809.0,NaN,NaN,21053,92807,CA,9275.0,0,920,2WD,NaN
4,2009-01-04,1500 RAM,QUAD CAB,2004,6949351.0,20577,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,76276,AMERICAN,LARGE TRUCK,CHRYSLER,4597.0,6357.0,5465.0,7366.0,4597.0,6357.0,5465.0,7366.0,NaN,NaN,20392,75236,TX,9450.0,0,920,2WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,0.0,68096,1,ADESA,4,SUZUKI,NaN,BAS,NaN,BLACK,AUTO,1.0,NaN,55109,NOT AMERICAN,MEDIUM SUV,OTHER,7057.0,7950.0,10843.0,11604.0,7050.0,8061.0,11114.0,11830.0,NaN,NaN,52598,28273,NC,7470.0,0,920,2WD,4D
58382,2010-12-30,MALIBU,SEDAN,2005,0.0,60722,1,MANHEIM,5,CHEVROLET,4C,CLA,CLASSIC,SILVER,AUTO,2.0,Covers,80072,AMERICAN,MEDIUM,GM,3806.0,5397.0,6708.0,8588.0,3643.0,5100.0,6908.0,8446.0,NaN,NaN,21047,92337,CA,4395.0,0,1243,NaN,4D
58383,2010-12-30,SPECTRA,SEDAN,2006,0.0,70450,0,ADESA,4,KIA,NaN,EX,NaN,SILVER,AUTO,2.0,Covers,43116,NOT AMERICAN,MEDIUM,OTHER,4894.0,6120.0,8136.0,9384.0,4384.0,5253.0,7160.0,8535.0,NO,GREEN,20740,32219,FL,5350.0,0,482,NaN,4D
58384,2010-12-30,TAURUS,SEDAN,2006,0.0,68098,0,ADESA,4,FORD,NaN,SE,NaN,GREY,AUTO,2.0,Covers,69853,AMERICAN,MEDIUM,FORD,3964.0,5281.0,7182.0,9569.0,4683.0,5973.0,7881.0,9271.0,NO,GREEN,52598,28273,NC,5145.0,0,825,NaN,4D


In [845]:
dff2 = df[Variable].groupby([df['PurchDate'], df['Model'], df['SubModel'], df['VehYear']]).size().reset_index()

In [846]:
dff2.rename(columns={Variable:'Count'}, inplace=True)
dff2

,PurchDate,Model,SubModel,VehYear,Count
0,2009-01-04,1500 RAM,QUAD CAB,2002,1
1,2009-01-04,1500 RAM,QUAD CAB,2003,1
2,2009-01-04,1500 RAM,QUAD CAB,2004,3
3,2009-01-04,1500 RAM,QUAD CAB,2005,1
4,2009-01-04,300,SEDAN,2006,3
...,...,...,...,...,...
40914,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,1
40915,2010-12-30,MALIBU,SEDAN,2005,1
40916,2010-12-30,SPECTRA,SEDAN,2006,1
40917,2010-12-30,TAURUS,SEDAN,2006,1


In [847]:
mergedDF = pd.merge(dff2, mergedDF, on=['PurchDate','Model', 'SubModel', 'VehYear'],  how='left')

In [848]:
mergedDF

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,2009-01-04,1500 RAM,QUAD CAB,2002,1,0.0,62472,0,ADESA,7,DODGE,5.9L,ST,NaN,BLUE,AUTO,1.0,Alloy,62209,AMERICAN,LARGE TRUCK,CHRYSLER,5872.0,7407.0,6842.0,8500.0,5872.0,7407.0,6842.0,8500.0,NaN,NaN,20234,85226,AZ,7940.0,0,1125,2WD,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,1,0.0,16342,0,MANHEIM,6,DODGE,5.7L,LAR,NaN,SILVER,AUTO,1.0,Alloy,79925,AMERICAN,LARGE TRUCK,CHRYSLER,7648.0,9717.0,8760.0,10994.0,7648.0,9717.0,8760.0,10994.0,NaN,NaN,19619,32824,FL,9400.0,0,1220,2WD,NaN
2,2009-01-04,1500 RAM,QUAD CAB,2004,3,6949351.0,46450,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,81296,AMERICAN,LARGE TRUCK,CHRYSLER,9620.0,14210.0,10890.0,15847.0,9620.0,14210.0,10890.0,15847.0,NaN,NaN,19638,33809,FL,9460.0,0,1893,4WD,NaN
3,2009-01-04,1500 RAM,QUAD CAB,2004,3,6949351.0,14101,0,MANHEIM,5,DODGE,5.7L,ST,NaN,BLACK,AUTO,3.0,Special,75251,AMERICAN,LARGE TRUCK,CHRYSLER,8496.0,10471.0,9676.0,11809.0,8496.0,10471.0,9676.0,11809.0,NaN,NaN,21053,92807,CA,9275.0,0,920,2WD,NaN
4,2009-01-04,1500 RAM,QUAD CAB,2004,3,6949351.0,20577,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,76276,AMERICAN,LARGE TRUCK,CHRYSLER,4597.0,6357.0,5465.0,7366.0,4597.0,6357.0,5465.0,7366.0,NaN,NaN,20392,75236,TX,9450.0,0,920,2WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,1,0.0,68096,1,ADESA,4,SUZUKI,NaN,BAS,NaN,BLACK,AUTO,1.0,NaN,55109,NOT AMERICAN,MEDIUM SUV,OTHER,7057.0,7950.0,10843.0,11604.0,7050.0,8061.0,11114.0,11830.0,NaN,NaN,52598,28273,NC,7470.0,0,920,2WD,4D
58382,2010-12-30,MALIBU,SEDAN,2005,1,0.0,60722,1,MANHEIM,5,CHEVROLET,4C,CLA,CLASSIC,SILVER,AUTO,2.0,Covers,80072,AMERICAN,MEDIUM,GM,3806.0,5397.0,6708.0,8588.0,3643.0,5100.0,6908.0,8446.0,NaN,NaN,21047,92337,CA,4395.0,0,1243,NaN,4D
58383,2010-12-30,SPECTRA,SEDAN,2006,1,0.0,70450,0,ADESA,4,KIA,NaN,EX,NaN,SILVER,AUTO,2.0,Covers,43116,NOT AMERICAN,MEDIUM,OTHER,4894.0,6120.0,8136.0,9384.0,4384.0,5253.0,7160.0,8535.0,NO,GREEN,20740,32219,FL,5350.0,0,482,NaN,4D
58384,2010-12-30,TAURUS,SEDAN,2006,1,0.0,68098,0,ADESA,4,FORD,NaN,SE,NaN,GREY,AUTO,2.0,Covers,69853,AMERICAN,MEDIUM,FORD,3964.0,5281.0,7182.0,9569.0,4683.0,5973.0,7881.0,9271.0,NO,GREEN,52598,28273,NC,5145.0,0,825,NaN,4D


In [849]:
nulli= mergedDF.loc[mergedDF['Varianza']<1]

In [850]:
nulli

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,2009-01-04,1500 RAM,QUAD CAB,2002,1,0.0,62472,0,ADESA,7,DODGE,5.9L,ST,NaN,BLUE,AUTO,1.0,Alloy,62209,AMERICAN,LARGE TRUCK,CHRYSLER,5872.0,7407.0,6842.0,8500.0,5872.0,7407.0,6842.0,8500.0,NaN,NaN,20234,85226,AZ,7940.0,0,1125,2WD,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,1,0.0,16342,0,MANHEIM,6,DODGE,5.7L,LAR,NaN,SILVER,AUTO,1.0,Alloy,79925,AMERICAN,LARGE TRUCK,CHRYSLER,7648.0,9717.0,8760.0,10994.0,7648.0,9717.0,8760.0,10994.0,NaN,NaN,19619,32824,FL,9400.0,0,1220,2WD,NaN
5,2009-01-04,1500 RAM,QUAD CAB,2005,1,0.0,16344,0,MANHEIM,4,DODGE,5.7L,SLT,NaN,SILVER,AUTO,2.0,Covers,78806,AMERICAN,LARGE TRUCK,CHRYSLER,10306.0,12558.0,11630.0,14063.0,10306.0,12558.0,11630.0,14063.0,NaN,NaN,19619,32824,FL,10000.0,0,920,2WD,NaN
6,2009-01-04,300,SEDAN,2006,3,0.0,62463,0,ADESA,3,CHRYSLER,2.7L MPI,BAS,NaN,GREY,AUTO,2.0,Covers,71624,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20234,85226,AZ,9505.0,0,1389,NaN,4D
7,2009-01-04,300,SEDAN,2006,3,0.0,20574,0,MANHEIM,3,CHRYSLER,2.7L MPI,BAS,NaN,BLUE,AUTO,1.0,NaN,64278,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20833,75236,TX,10050.0,0,1215,NaN,4D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,1,0.0,68096,1,ADESA,4,SUZUKI,NaN,BAS,NaN,BLACK,AUTO,1.0,NaN,55109,NOT AMERICAN,MEDIUM SUV,OTHER,7057.0,7950.0,10843.0,11604.0,7050.0,8061.0,11114.0,11830.0,NaN,NaN,52598,28273,NC,7470.0,0,920,2WD,4D
58382,2010-12-30,MALIBU,SEDAN,2005,1,0.0,60722,1,MANHEIM,5,CHEVROLET,4C,CLA,CLASSIC,SILVER,AUTO,2.0,Covers,80072,AMERICAN,MEDIUM,GM,3806.0,5397.0,6708.0,8588.0,3643.0,5100.0,6908.0,8446.0,NaN,NaN,21047,92337,CA,4395.0,0,1243,NaN,4D
58383,2010-12-30,SPECTRA,SEDAN,2006,1,0.0,70450,0,ADESA,4,KIA,NaN,EX,NaN,SILVER,AUTO,2.0,Covers,43116,NOT AMERICAN,MEDIUM,OTHER,4894.0,6120.0,8136.0,9384.0,4384.0,5253.0,7160.0,8535.0,NO,GREEN,20740,32219,FL,5350.0,0,482,NaN,4D
58384,2010-12-30,TAURUS,SEDAN,2006,1,0.0,68098,0,ADESA,4,FORD,NaN,SE,NaN,GREY,AUTO,2.0,Covers,69853,AMERICAN,MEDIUM,FORD,3964.0,5281.0,7182.0,9569.0,4683.0,5973.0,7881.0,9271.0,NO,GREEN,52598,28273,NC,5145.0,0,825,NaN,4D


In [851]:
nulli= mergedDF.loc[mergedDF['Varianza']<=250000]

In [852]:
nulli

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,2009-01-04,1500 RAM,QUAD CAB,2002,1,0.0,62472,0,ADESA,7,DODGE,5.9L,ST,NaN,BLUE,AUTO,1.0,Alloy,62209,AMERICAN,LARGE TRUCK,CHRYSLER,5872.0,7407.0,6842.0,8500.0,5872.0,7407.0,6842.0,8500.0,NaN,NaN,20234,85226,AZ,7940.0,0,1125,2WD,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,1,0.0,16342,0,MANHEIM,6,DODGE,5.7L,LAR,NaN,SILVER,AUTO,1.0,Alloy,79925,AMERICAN,LARGE TRUCK,CHRYSLER,7648.0,9717.0,8760.0,10994.0,7648.0,9717.0,8760.0,10994.0,NaN,NaN,19619,32824,FL,9400.0,0,1220,2WD,NaN
5,2009-01-04,1500 RAM,QUAD CAB,2005,1,0.0,16344,0,MANHEIM,4,DODGE,5.7L,SLT,NaN,SILVER,AUTO,2.0,Covers,78806,AMERICAN,LARGE TRUCK,CHRYSLER,10306.0,12558.0,11630.0,14063.0,10306.0,12558.0,11630.0,14063.0,NaN,NaN,19619,32824,FL,10000.0,0,920,2WD,NaN
6,2009-01-04,300,SEDAN,2006,3,0.0,62463,0,ADESA,3,CHRYSLER,2.7L MPI,BAS,NaN,GREY,AUTO,2.0,Covers,71624,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20234,85226,AZ,9505.0,0,1389,NaN,4D
7,2009-01-04,300,SEDAN,2006,3,0.0,20574,0,MANHEIM,3,CHRYSLER,2.7L MPI,BAS,NaN,BLUE,AUTO,1.0,NaN,64278,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20833,75236,TX,10050.0,0,1215,NaN,4D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,1,0.0,68096,1,ADESA,4,SUZUKI,NaN,BAS,NaN,BLACK,AUTO,1.0,NaN,55109,NOT AMERICAN,MEDIUM SUV,OTHER,7057.0,7950.0,10843.0,11604.0,7050.0,8061.0,11114.0,11830.0,NaN,NaN,52598,28273,NC,7470.0,0,920,2WD,4D
58382,2010-12-30,MALIBU,SEDAN,2005,1,0.0,60722,1,MANHEIM,5,CHEVROLET,4C,CLA,CLASSIC,SILVER,AUTO,2.0,Covers,80072,AMERICAN,MEDIUM,GM,3806.0,5397.0,6708.0,8588.0,3643.0,5100.0,6908.0,8446.0,NaN,NaN,21047,92337,CA,4395.0,0,1243,NaN,4D
58383,2010-12-30,SPECTRA,SEDAN,2006,1,0.0,70450,0,ADESA,4,KIA,NaN,EX,NaN,SILVER,AUTO,2.0,Covers,43116,NOT AMERICAN,MEDIUM,OTHER,4894.0,6120.0,8136.0,9384.0,4384.0,5253.0,7160.0,8535.0,NO,GREEN,20740,32219,FL,5350.0,0,482,NaN,4D
58384,2010-12-30,TAURUS,SEDAN,2006,1,0.0,68098,0,ADESA,4,FORD,NaN,SE,NaN,GREY,AUTO,2.0,Covers,69853,AMERICAN,MEDIUM,FORD,3964.0,5281.0,7182.0,9569.0,4683.0,5973.0,7881.0,9271.0,NO,GREEN,52598,28273,NC,5145.0,0,825,NaN,4D


In [853]:
nulli= mergedDF.loc[mergedDF['Varianza']>250000]

## 5629 macchine non concordano con la definizione di MMR

In [854]:
nulli

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
2,2009-01-04,1500 RAM,QUAD CAB,2004,3,6949351.00,46450,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,81296,AMERICAN,LARGE TRUCK,CHRYSLER,9620.0,14210.0,10890.0,15847.0,9620.0,14210.0,10890.0,15847.0,NaN,NaN,19638,33809,FL,9460.0,0,1893,4WD,NaN
3,2009-01-04,1500 RAM,QUAD CAB,2004,3,6949351.00,14101,0,MANHEIM,5,DODGE,5.7L,ST,NaN,BLACK,AUTO,3.0,Special,75251,AMERICAN,LARGE TRUCK,CHRYSLER,8496.0,10471.0,9676.0,11809.0,8496.0,10471.0,9676.0,11809.0,NaN,NaN,21053,92807,CA,9275.0,0,920,2WD,NaN
4,2009-01-04,1500 RAM,QUAD CAB,2004,3,6949351.00,20577,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,76276,AMERICAN,LARGE TRUCK,CHRYSLER,4597.0,6357.0,5465.0,7366.0,4597.0,6357.0,5465.0,7366.0,NaN,NaN,20392,75236,TX,9450.0,0,920,2WD,NaN
24,2009-01-04,CARAVAN,MINIVAN,2005,4,294818.25,46448,0,MANHEIM,4,DODGE,3.8L,SXT,NaN,NOT AVAIL,AUTO,1.0,Alloy,81955,AMERICAN,VAN,CHRYSLER,6002.0,7884.0,6982.0,9015.0,6002.0,7884.0,6982.0,9015.0,NaN,NaN,19638,33809,FL,5830.0,0,1763,2WD,NaN
25,2009-01-04,CARAVAN,MINIVAN,2005,4,294818.25,62464,1,ADESA,4,DODGE,3.3L,SE,NaN,WHITE,AUTO,1.0,Alloy,77746,AMERICAN,VAN,CHRYSLER,4856.0,6312.0,5744.0,7317.0,4856.0,6312.0,5744.0,7317.0,NaN,NaN,835,85226,AZ,5675.0,0,1623,2WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58233,2010-12-22,IMPALA,SEDAN,2005,3,442537.00,2980,1,OTHER,5,CHEVROLET,NaN,LS,NaN,WHITE,AUTO,1.0,NaN,108330,AMERICAN,LARGE,GM,4947.0,6287.0,8295.0,10038.0,4795.0,6113.0,7968.0,9287.0,NaN,NaN,18880,35004,AL,4100.0,0,2571,NaN,4D
58234,2010-12-22,IMPALA,SEDAN,2005,3,442537.00,23757,0,MANHEIM,5,CHEVROLET,NaN,BAS,NaN,GOLD,AUTO,2.0,Covers,84367,AMERICAN,LARGE,GM,3782.0,5024.0,6780.0,8395.0,4013.0,5328.0,6674.0,8360.0,NO,GREEN,23359,92807,CA,5330.0,0,2152,NaN,4D
58235,2010-12-22,IMPALA,SEDAN,2005,3,442537.00,10273,0,MANHEIM,5,CHEVROLET,NaN,BAS,NaN,SILVER,AUTO,2.0,Covers,93386,AMERICAN,LARGE,GM,3808.0,5058.0,6814.0,8439.0,4032.0,5391.0,7072.0,8797.0,NO,GREEN,21047,94544,CA,5220.0,0,2351,NaN,4D
58275,2010-12-22,SABLE,SEDAN,2002,2,252760.50,52397,0,MANHEIM,8,MERCURY,NaN,GS,NaN,GOLD,AUTO,2.0,Covers,109662,AMERICAN,MEDIUM,FORD,2183.0,3047.0,4244.0,5604.0,2204.0,3214.0,4973.0,5740.0,NO,GREEN,53245,78219,TX,4235.0,0,1430,NaN,4D


In [855]:
varbassa = mergedDF.loc[mergedDF['Varianza']<250000]

In [856]:
varbassa

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,2009-01-04,1500 RAM,QUAD CAB,2002,1,0.0,62472,0,ADESA,7,DODGE,5.9L,ST,NaN,BLUE,AUTO,1.0,Alloy,62209,AMERICAN,LARGE TRUCK,CHRYSLER,5872.0,7407.0,6842.0,8500.0,5872.0,7407.0,6842.0,8500.0,NaN,NaN,20234,85226,AZ,7940.0,0,1125,2WD,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,1,0.0,16342,0,MANHEIM,6,DODGE,5.7L,LAR,NaN,SILVER,AUTO,1.0,Alloy,79925,AMERICAN,LARGE TRUCK,CHRYSLER,7648.0,9717.0,8760.0,10994.0,7648.0,9717.0,8760.0,10994.0,NaN,NaN,19619,32824,FL,9400.0,0,1220,2WD,NaN
5,2009-01-04,1500 RAM,QUAD CAB,2005,1,0.0,16344,0,MANHEIM,4,DODGE,5.7L,SLT,NaN,SILVER,AUTO,2.0,Covers,78806,AMERICAN,LARGE TRUCK,CHRYSLER,10306.0,12558.0,11630.0,14063.0,10306.0,12558.0,11630.0,14063.0,NaN,NaN,19619,32824,FL,10000.0,0,920,2WD,NaN
6,2009-01-04,300,SEDAN,2006,3,0.0,62463,0,ADESA,3,CHRYSLER,2.7L MPI,BAS,NaN,GREY,AUTO,2.0,Covers,71624,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20234,85226,AZ,9505.0,0,1389,NaN,4D
7,2009-01-04,300,SEDAN,2006,3,0.0,20574,0,MANHEIM,3,CHRYSLER,2.7L MPI,BAS,NaN,BLUE,AUTO,1.0,NaN,64278,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20833,75236,TX,10050.0,0,1215,NaN,4D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,1,0.0,68096,1,ADESA,4,SUZUKI,NaN,BAS,NaN,BLACK,AUTO,1.0,NaN,55109,NOT AMERICAN,MEDIUM SUV,OTHER,7057.0,7950.0,10843.0,11604.0,7050.0,8061.0,11114.0,11830.0,NaN,NaN,52598,28273,NC,7470.0,0,920,2WD,4D
58382,2010-12-30,MALIBU,SEDAN,2005,1,0.0,60722,1,MANHEIM,5,CHEVROLET,4C,CLA,CLASSIC,SILVER,AUTO,2.0,Covers,80072,AMERICAN,MEDIUM,GM,3806.0,5397.0,6708.0,8588.0,3643.0,5100.0,6908.0,8446.0,NaN,NaN,21047,92337,CA,4395.0,0,1243,NaN,4D
58383,2010-12-30,SPECTRA,SEDAN,2006,1,0.0,70450,0,ADESA,4,KIA,NaN,EX,NaN,SILVER,AUTO,2.0,Covers,43116,NOT AMERICAN,MEDIUM,OTHER,4894.0,6120.0,8136.0,9384.0,4384.0,5253.0,7160.0,8535.0,NO,GREEN,20740,32219,FL,5350.0,0,482,NaN,4D
58384,2010-12-30,TAURUS,SEDAN,2006,1,0.0,68098,0,ADESA,4,FORD,NaN,SE,NaN,GREY,AUTO,2.0,Covers,69853,AMERICAN,MEDIUM,FORD,3964.0,5281.0,7182.0,9569.0,4683.0,5973.0,7881.0,9271.0,NO,GREEN,52598,28273,NC,5145.0,0,825,NaN,4D


In [857]:
contvarbassa = varbassa.loc[varbassa['Count']>5]

In [858]:
contvarbassa

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
177,2009-01-05,AVENGER,SEDAN,2008,12,0.000000,55791,0,OTHER,1,DODGE,4C 2.4L,SE,NaN,BLACK,AUTO,2.0,Covers,38212,AMERICAN,MEDIUM,CHRYSLER,8196.0,9044.0,9352.0,10268.0,8196.0,9044.0,9352.0,10268.0,NaN,NaN,99761,33073,FL,8200.0,0,762,NaN,4D
178,2009-01-05,AVENGER,SEDAN,2008,12,0.000000,55795,0,OTHER,1,DODGE,4C 2.4L,SE,NaN,BLACK,AUTO,2.0,Covers,38085,AMERICAN,MEDIUM,CHRYSLER,8196.0,9044.0,9352.0,10268.0,8196.0,9044.0,9352.0,10268.0,NaN,NaN,99761,33073,FL,8200.0,0,762,NaN,4D
179,2009-01-05,AVENGER,SEDAN,2008,12,0.000000,55773,0,OTHER,1,DODGE,4C 2.4L,SE,NaN,BLACK,AUTO,2.0,Covers,38978,AMERICAN,MEDIUM,CHRYSLER,8196.0,9044.0,9352.0,10268.0,8196.0,9044.0,9352.0,10268.0,NaN,NaN,99761,33073,FL,8200.0,0,762,NaN,4D
180,2009-01-05,AVENGER,SEDAN,2008,12,0.000000,55789,0,OTHER,1,DODGE,4C 2.4L,SE,NaN,BLUE,AUTO,2.0,Covers,39066,AMERICAN,MEDIUM,CHRYSLER,8196.0,9044.0,9352.0,10268.0,8196.0,9044.0,9352.0,10268.0,NaN,NaN,99761,33073,FL,8200.0,0,762,NaN,4D
181,2009-01-05,AVENGER,SEDAN,2008,12,0.000000,55772,0,OTHER,1,DODGE,4C 2.4L,SE,NaN,WHITE,AUTO,2.0,Covers,47529,AMERICAN,MEDIUM,CHRYSLER,8196.0,9044.0,9352.0,10268.0,8196.0,9044.0,9352.0,10268.0,NaN,NaN,99761,33073,FL,8200.0,0,834,NaN,4D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57697,2010-12-15,IMPALA,SEDAN,2007,6,1664.266667,44610,1,MANHEIM,3,CHEVROLET,NaN,LS,NaN,GOLD,AUTO,2.0,Covers,69306,AMERICAN,LARGE,GM,7345.0,8489.0,10022.0,11498.0,7078.0,8282.0,10791.0,12229.0,NaN,NaN,99750,32124,FL,8400.0,0,1703,NaN,4D
57698,2010-12-15,IMPALA,SEDAN,2007,6,1664.266667,54981,0,OTHER,3,CHEVROLET,NaN,LS,NaN,MAROON,AUTO,2.0,Covers,91345,AMERICAN,LARGE,GM,7424.0,8595.0,10124.0,11634.0,7318.0,8582.0,11169.0,12449.0,NaN,NaN,22808,73108,OK,7300.0,0,2351,NaN,4D
57699,2010-12-15,IMPALA,SEDAN,2007,6,1664.266667,44606,0,MANHEIM,3,CHEVROLET,NaN,LS,NaN,GOLD,AUTO,2.0,Covers,66794,AMERICAN,LARGE,GM,7345.0,8489.0,10022.0,11498.0,6814.0,7885.0,10481.0,11725.0,NaN,NaN,99750,32124,FL,8200.0,0,1703,NaN,4D
57700,2010-12-15,IMPALA,SEDAN,2007,6,1664.266667,54976,0,OTHER,3,CHEVROLET,NaN,LS,NaN,SILVER,AUTO,1.0,Alloy,73369,AMERICAN,LARGE,GM,7424.0,8595.0,10124.0,11634.0,7757.0,9024.0,11137.0,12326.0,NO,GREEN,22808,73108,OK,7300.0,0,1974,NaN,4D


In [859]:
varbassa['Count'].max()

63

In [860]:
dffsenzadata

,Model,SubModel,VehYear,MMRAcquisitionAuctionAveragePrice
0,1500 RAM,EXT CAB,2001,1.705280e+05
1,1500 RAM,MEGA CAB,2006,2.279093e+05
2,1500 RAM,QUAD CAB,2002,1.060502e+06
3,1500 RAM,QUAD CAB,2003,1.378630e+06
4,1500 RAM,QUAD CAB,2004,2.498145e+06
...,...,...,...,...
1383,YUKON XL,UTILITY,2002,1.549913e+06
1384,YUKON XL,UTILITY,2003,6.630052e+05
1385,YUKON XL,UTILITY,2004,3.928118e+05
1386,YUKON XL,UTILITY,2005,NaN


## Analisi dffsenzadata

In [861]:
dffsenzadata.rename(columns={Variable:'Varianza'}, inplace=True)
dffsenzadata

,Model,SubModel,VehYear,Varianza
0,1500 RAM,EXT CAB,2001,1.705280e+05
1,1500 RAM,MEGA CAB,2006,2.279093e+05
2,1500 RAM,QUAD CAB,2002,1.060502e+06
3,1500 RAM,QUAD CAB,2003,1.378630e+06
4,1500 RAM,QUAD CAB,2004,2.498145e+06
...,...,...,...,...
1383,YUKON XL,UTILITY,2002,1.549913e+06
1384,YUKON XL,UTILITY,2003,6.630052e+05
1385,YUKON XL,UTILITY,2004,3.928118e+05
1386,YUKON XL,UTILITY,2005,NaN


In [862]:
dffsenzadata['Varianza'].isnull().sum()

256

In [863]:
dffsenzadata['Varianza'] = dffsenzadata['Varianza'].fillna(0)

## Varianza dffsenzadata
all'incirca il 35% del database ha una varianza inferiore alla soglia scelta (250000) mentre il 0.4% ha una varianza pari a 0

In [864]:
mergedDF2 = pd.merge(dffsenzadata, df, on=['Model', 'SubModel', 'VehYear'],  how='left')

In [865]:
mergedDF2

,Model,SubModel,VehYear,Varianza,RefId,IsBadBuy,PurchDate,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,1500 RAM,EXT CAB,2001,1.705280e+05,68125,0,2009-01-29,ADESA,8,DODGE,5.9L,LAR,LARAMIE,BLACK,AUTO,1.0,Alloy,87847,AMERICAN,LARGE TRUCK,CHRYSLER,2608.0,3516.0,3317.0,4297.0,3392.0,4436.0,4163.0,5291.0,NaN,NaN,3453,80817,CO,4480.0,0,1283,2WD,4D
1,1500 RAM,EXT CAB,2001,1.705280e+05,61608,1,2009-07-28,ADESA,8,DODGE,5.2L,LAR,LARAMIE,BLUE,AUTO,1.0,NaN,55117,AMERICAN,LARGE TRUCK,CHRYSLER,3192.0,4269.0,3947.0,5111.0,3192.0,4269.0,3947.0,5111.0,NaN,NaN,23359,91752,CA,4315.0,0,1054,2WD,4D
2,1500 RAM,MEGA CAB,2006,2.279093e+05,50261,0,2009-02-25,MANHEIM,3,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,72728,AMERICAN,LARGE TRUCK,CHRYSLER,12423.0,15017.0,13917.0,16718.0,13721.0,15229.0,15319.0,16947.0,NaN,NaN,19662,78219,TX,10835.0,0,920,2WD,NaN
3,1500 RAM,MEGA CAB,2006,2.279093e+05,33703,0,2009-01-12,MANHEIM,3,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,80650,AMERICAN,LARGE TRUCK,CHRYSLER,13271.0,15154.0,14833.0,16866.0,12982.0,14842.0,18542.0,20697.0,NaN,NaN,19619,34761,FL,11515.0,0,983,2WD,NaN
4,1500 RAM,MEGA CAB,2006,2.279093e+05,39145,0,2010-03-18,MANHEIM,4,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,91710,AMERICAN,LARGE TRUCK,CHRYSLER,13227.0,16033.0,17410.0,19843.0,14571.0,15835.0,18891.0,20220.0,NaN,NaN,17212,76040,TX,10550.0,0,1053,2WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,ZEPHYR,SEDAN,2006,3.236414e+06,70031,0,2010-07-14,ADESA,4,LINCOLN,NaN,BAS,NaN,RED,AUTO,1.0,Alloy,70797,AMERICAN,SPECIALTY,FORD,12532.0,14479.0,16113.0,17482.0,11528.0,14189.0,15602.0,18140.0,NaN,NaN,21053,85226,AZ,8615.0,0,1272,NaN,4D
58382,ZEPHYR,SEDAN,2006,3.236414e+06,25215,0,2010-02-18,OTHER,4,LINCOLN,NaN,BAS,NaN,GOLD,AUTO,1.0,Alloy,82286,AMERICAN,SPECIALTY,FORD,12341.0,13964.0,16405.0,17844.0,13115.0,14984.0,16693.0,18439.0,NaN,NaN,3453,80229,CO,8675.0,0,1373,NaN,4D
58383,ZEPHYR,SEDAN,2006,3.236414e+06,32556,0,2009-03-03,MANHEIM,3,LINCOLN,NaN,BAS,NaN,BEIGE,AUTO,1.0,Alloy,68976,AMERICAN,SPECIALTY,FORD,13765.0,15297.0,15366.0,17021.0,13981.0,15490.0,15599.0,17229.0,NaN,NaN,3453,80022,CO,9215.0,0,1118,NaN,4D
58384,ZEPHYR,SEDAN,2006,3.236414e+06,13079,0,2010-10-18,MANHEIM,4,LINCOLN,NaN,BAS,NaN,BEIGE,AUTO,1.0,Alloy,76988,AMERICAN,SPECIALTY,FORD,10448.0,12558.0,14819.0,17078.0,10098.0,12489.0,14519.0,16362.0,NO,GREEN,17675,27542,NC,10000.0,0,1272,NaN,4D


In [866]:
nulli= mergedDF2.loc[mergedDF2['Varianza']<1]

In [867]:
nulli

,Model,SubModel,VehYear,Varianza,RefId,IsBadBuy,PurchDate,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
926,1500 RAM,REG CAB,2008,0.0,37121,0,2010-06-30,MANHEIM,2,DODGE,3.7L,ST,NaN,RED,AUTO,2.0,Covers,67650,AMERICAN,LARGE TRUCK,CHRYSLER,10001.0,12108.0,15341.0,16741.0,8252.0,8661.0,14080.0,14307.0,NaN,NaN,25100,75236,TX,7940.0,0,533,2WD,NaN
927,1500 SIERRA,CREW CAB,2004,0.0,48253,0,2009-10-02,OTHER,5,GMC,5.3L,SLE,NaN,BEIGE,AUTO,1.0,Alloy,87442,AMERICAN,LARGE TRUCK,GM,8541.0,10491.0,9724.0,11830.0,9160.0,10126.0,10393.0,11436.0,NaN,NaN,19662,70401,LA,10500.0,0,1243,NaN,NaN
1525,1500HD SILVERADO,CREW CAB,2006,0.0,22472,0,2009-06-18,MANHEIM,3,CHEVROLET,6.0L,LT,NaN,WHITE,AUTO,1.0,Alloy,88928,AMERICAN,LARGE TRUCK,GM,12730.0,16220.0,14248.0,18018.0,10909.0,13903.0,12282.0,15515.0,NaN,NaN,17212,76040,TX,11790.0,0,1243,NaN,NaN
1535,2500 RAM,QUAD CAB,2006,0.0,49328,1,2010-10-06,OTHER,4,DODGE,5.7L,LAR,NaN,WHITE,AUTO,1.0,Alloy,86735,AMERICAN,LARGE TRUCK,CHRYSLER,10061.0,12975.0,15103.0,18153.0,8086.0,11222.0,12800.0,16319.0,NaN,NaN,20833,73108,OK,10955.0,0,983,2WD,NaN
1536,2500 RAM,QUAD CAB,2007,0.0,5821,0,2010-12-07,MANHEIM,3,DODGE,5.7L,LAR,BIG HORN,RED,AUTO,2.0,Covers,92488,AMERICAN,LARGE TRUCK,CHRYSLER,18171.0,22252.0,23613.0,27513.0,18320.0,20843.0,23451.0,26375.0,NaN,NaN,99750,85040,AZ,11000.0,0,2062,4WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58054,XL-7,UTILITY SPORT,2003,0.0,37819,0,2010-10-06,MANHEIM,7,SUZUKI,NaN,TOU,NaN,WHITE,AUTO,1.0,Alloy,58265,NOT AMERICAN,MEDIUM SUV,OTHER,5145.0,6267.0,8490.0,9629.0,5674.0,6809.0,9034.0,10306.0,NaN,NaN,22916,89120,NV,7620.0,0,1551,4WD,4D
58272,YARIS,HATCHBACK,2009,0.0,31196,0,2010-09-30,OTHER,1,TOYOTA,NaN,BAS,NaN,BLACK,AUTO,2.0,Covers,49368,NOT AMERICAN,COMPACT,OTHER,8251.0,8859.0,10577.0,11179.0,7885.0,8618.0,9981.0,10909.0,NaN,NaN,99761,85284,AZ,7550.0,0,482,NaN,2D
58273,YARIS,HATCHBACK,2009,0.0,31200,0,2010-09-30,OTHER,1,TOYOTA,NaN,BAS,NaN,WHITE,AUTO,2.0,Covers,47773,NOT AMERICAN,COMPACT,OTHER,8251.0,8859.0,10577.0,11179.0,8245.0,8767.0,10428.0,11089.0,NaN,NaN,99761,85284,AZ,7550.0,0,482,NaN,2D
58288,YARIS,SEDAN,2009,0.0,31155,0,2010-09-30,OTHER,1,TOYOTA,NaN,BAS,NaN,SILVER,AUTO,2.0,Covers,49459,NOT AMERICAN,COMPACT,OTHER,9054.0,9696.0,11372.0,12008.0,9173.0,10044.0,11329.0,12123.0,NaN,NaN,99761,85284,AZ,7550.0,0,482,NaN,4D


In [868]:
nulli= mergedDF2.loc[mergedDF2['Varianza']<=250000]

In [869]:
nulli

,Model,SubModel,VehYear,Varianza,RefId,IsBadBuy,PurchDate,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,1500 RAM,EXT CAB,2001,170528.000000,68125,0,2009-01-29,ADESA,8,DODGE,5.9L,LAR,LARAMIE,BLACK,AUTO,1.0,Alloy,87847,AMERICAN,LARGE TRUCK,CHRYSLER,2608.0,3516.0,3317.0,4297.0,3392.0,4436.0,4163.0,5291.0,NaN,NaN,3453,80817,CO,4480.0,0,1283,2WD,4D
1,1500 RAM,EXT CAB,2001,170528.000000,61608,1,2009-07-28,ADESA,8,DODGE,5.2L,LAR,LARAMIE,BLUE,AUTO,1.0,NaN,55117,AMERICAN,LARGE TRUCK,CHRYSLER,3192.0,4269.0,3947.0,5111.0,3192.0,4269.0,3947.0,5111.0,NaN,NaN,23359,91752,CA,4315.0,0,1054,2WD,4D
2,1500 RAM,MEGA CAB,2006,227909.333333,50261,0,2009-02-25,MANHEIM,3,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,72728,AMERICAN,LARGE TRUCK,CHRYSLER,12423.0,15017.0,13917.0,16718.0,13721.0,15229.0,15319.0,16947.0,NaN,NaN,19662,78219,TX,10835.0,0,920,2WD,NaN
3,1500 RAM,MEGA CAB,2006,227909.333333,33703,0,2009-01-12,MANHEIM,3,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,80650,AMERICAN,LARGE TRUCK,CHRYSLER,13271.0,15154.0,14833.0,16866.0,12982.0,14842.0,18542.0,20697.0,NaN,NaN,19619,34761,FL,11515.0,0,983,2WD,NaN
4,1500 RAM,MEGA CAB,2006,227909.333333,39145,0,2010-03-18,MANHEIM,4,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,91710,AMERICAN,LARGE TRUCK,CHRYSLER,13227.0,16033.0,17410.0,19843.0,14571.0,15835.0,18891.0,20220.0,NaN,NaN,17212,76040,TX,10550.0,0,1053,2WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58291,YUKON,UTILITY,2001,64772.166667,6867,0,2010-08-31,MANHEIM,9,GMC,V8,SLE,NaN,GREY,AUTO,1.0,Alloy,82353,AMERICAN,LARGE SUV,GM,6240.0,7634.0,9211.0,10817.0,6117.0,7519.0,9017.0,10426.0,NaN,NaN,835,85040,AZ,8370.0,0,2063,2WD,4D
58292,YUKON,UTILITY,2001,64772.166667,9727,1,2009-04-21,MANHEIM,8,GMC,V8 5.3L,DEN,NaN,GOLD,AUTO,1.0,Alloy,82462,AMERICAN,LARGE SUV,GM,6040.0,7268.0,7023.0,8349.0,6040.0,7268.0,7023.0,8349.0,NaN,NaN,20740,21075,MD,9570.0,0,2063,4WD,4D
58293,YUKON,UTILITY,2001,64772.166667,49416,0,2010-10-14,OTHER,9,GMC,V8,DEN,NaN,MAROON,AUTO,1.0,Alloy,94235,AMERICAN,LARGE SUV,GM,6561.0,7990.0,9946.0,11723.0,5443.0,6795.0,8553.0,9946.0,NaN,NaN,17212,73108,OK,8290.0,0,2218,4WD,4D
58294,YUKON,UTILITY,2001,64772.166667,51379,0,2009-05-20,MANHEIM,8,GMC,V8 5.3L,DEN,NaN,SILVER,AUTO,1.0,Alloy,82327,AMERICAN,LARGE SUV,GM,6040.0,7268.0,7023.0,8349.0,6040.0,7268.0,7023.0,8349.0,NaN,NaN,3453,80011,CO,7805.0,0,2063,4WD,4D


In [870]:
nulli= mergedDF2.loc[mergedDF2['Varianza']>250000]

## 37000 macchine con rispettano la soglia

In [871]:
nulli

,Model,SubModel,VehYear,Varianza,RefId,IsBadBuy,PurchDate,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
5,1500 RAM,QUAD CAB,2002,1.060502e+06,63315,0,2010-11-30,MANHEIM,8,DODGE,4.7L,ST,NaN,GOLD,AUTO,1.0,Alloy,84794,AMERICAN,LARGE TRUCK,CHRYSLER,5890.0,7501.0,8971.0,10374.0,5171.0,6304.0,8729.0,9573.0,NO,GREEN,52598,28625,NC,9110.0,0,1283,2WD,NaN
6,1500 RAM,QUAD CAB,2002,1.060502e+06,36501,0,2009-02-10,MANHEIM,7,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,89552,AMERICAN,LARGE TRUCK,CHRYSLER,3559.0,3997.0,4344.0,4817.0,3942.0,5594.0,6841.0,9425.0,NaN,NaN,17212,75236,TX,7650.0,0,1283,2WD,NaN
7,1500 RAM,QUAD CAB,2002,1.060502e+06,64268,0,2010-03-17,ADESA,8,DODGE,5.9L,ST,NaN,GREY,AUTO,1.0,Alloy,78656,AMERICAN,LARGE TRUCK,CHRYSLER,7040.0,8026.0,10864.0,11760.0,7000.0,7997.0,11254.0,12360.0,NaN,NaN,835,85226,AZ,10770.0,0,2040,4WD,NaN
8,1500 RAM,QUAD CAB,2002,1.060502e+06,23228,1,2010-12-08,MANHEIM,8,DODGE,4.7L,ST,NaN,MAROON,AUTO,1.0,Alloy,87788,AMERICAN,LARGE TRUCK,CHRYSLER,5286.0,6558.0,8768.0,10216.0,5503.0,6875.0,8901.0,10046.0,NaN,NaN,18880,29532,SC,9110.0,0,1283,2WD,NaN
9,1500 RAM,QUAD CAB,2002,1.060502e+06,30483,0,2010-09-22,MANHEIM,8,DODGE,5.9L,ST,NaN,GREY,AUTO,1.0,Alloy,78709,AMERICAN,LARGE TRUCK,CHRYSLER,5350.0,6706.0,8589.0,9640.0,5152.0,6437.0,9125.0,10242.0,NO,GREEN,19619,32824,FL,8850.0,0,1220,2WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,ZEPHYR,SEDAN,2006,3.236414e+06,70031,0,2010-07-14,ADESA,4,LINCOLN,NaN,BAS,NaN,RED,AUTO,1.0,Alloy,70797,AMERICAN,SPECIALTY,FORD,12532.0,14479.0,16113.0,17482.0,11528.0,14189.0,15602.0,18140.0,NaN,NaN,21053,85226,AZ,8615.0,0,1272,NaN,4D
58382,ZEPHYR,SEDAN,2006,3.236414e+06,25215,0,2010-02-18,OTHER,4,LINCOLN,NaN,BAS,NaN,GOLD,AUTO,1.0,Alloy,82286,AMERICAN,SPECIALTY,FORD,12341.0,13964.0,16405.0,17844.0,13115.0,14984.0,16693.0,18439.0,NaN,NaN,3453,80229,CO,8675.0,0,1373,NaN,4D
58383,ZEPHYR,SEDAN,2006,3.236414e+06,32556,0,2009-03-03,MANHEIM,3,LINCOLN,NaN,BAS,NaN,BEIGE,AUTO,1.0,Alloy,68976,AMERICAN,SPECIALTY,FORD,13765.0,15297.0,15366.0,17021.0,13981.0,15490.0,15599.0,17229.0,NaN,NaN,3453,80022,CO,9215.0,0,1118,NaN,4D
58384,ZEPHYR,SEDAN,2006,3.236414e+06,13079,0,2010-10-18,MANHEIM,4,LINCOLN,NaN,BAS,NaN,BEIGE,AUTO,1.0,Alloy,76988,AMERICAN,SPECIALTY,FORD,10448.0,12558.0,14819.0,17078.0,10098.0,12489.0,14519.0,16362.0,NO,GREEN,17675,27542,NC,10000.0,0,1272,NaN,4D


In [872]:
mergedDF2['Varianza'].describe()

count    5.838600e+04
mean     5.821435e+05
std      1.006289e+06
min      0.000000e+00
25%      1.943570e+05
50%      3.493951e+05
75%      6.612594e+05
max      1.265959e+08
Name: Varianza, dtype: float64

In [873]:
mergedDF['Varianza'].describe()

count    5.838600e+04
mean     1.302233e+05
std      6.592548e+05
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      7.290000e+02
max      3.077986e+07
Name: Varianza, dtype: float64

# Analisi retail price

In [887]:
Variable='MMRAcquisitionRetailAveragePrice'

## dff è il dataset raggruppato con la definizione corretta di MMR

In [888]:
dff = df[Variable].groupby([df['PurchDate'], df['Model'], df['SubModel'], df['VehYear']]).var().reset_index()

## dffsenzadata è il dataset raggruppato senza tener conto della data di stima del MMR

In [890]:
dffsenzadata = df[Variable].groupby([df['Model'], df['SubModel'], df['VehYear']]).var().reset_index()

In [891]:
dff.rename(columns={Variable:'Varianza'}, inplace=True)
dff

,PurchDate,Model,SubModel,VehYear,Varianza
0,2009-01-04,1500 RAM,QUAD CAB,2002,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,NaN
2,2009-01-04,1500 RAM,QUAD CAB,2004,8106157.0
3,2009-01-04,1500 RAM,QUAD CAB,2005,NaN
4,2009-01-04,300,SEDAN,2006,0.0
...,...,...,...,...,...
40914,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,NaN
40915,2010-12-30,MALIBU,SEDAN,2005,NaN
40916,2010-12-30,SPECTRA,SEDAN,2006,NaN
40917,2010-12-30,TAURUS,SEDAN,2006,NaN


In [892]:
dff['Varianza'].isnull().sum()

31680

Una varianza pari a null indica che il raggruppamento corrisponde a una sola macchina o sono presenti solo macchine con valori null per l'MMR

In [893]:
dff['Varianza'] = dff['Varianza'].fillna(0)

# Varianza
all'incirca il 88% del database ha una varianza inferiore alla soglia scelta (250000) mentre il 71 % ha una varianza pari a 0 

In [894]:
dff.loc[dff["Varianza"]<1]["Varianza"]

0        0.0
1        0.0
3        0.0
4        0.0
5        0.0
        ... 
40914    0.0
40915    0.0
40916    0.0
40917    0.0
40918    0.0
Name: Varianza, Length: 35253, dtype: float64

In [895]:
dff.loc[dff["Varianza"]<=250000]["Varianza"]

0        0.0
1        0.0
3        0.0
4        0.0
5        0.0
        ... 
40914    0.0
40915    0.0
40916    0.0
40917    0.0
40918    0.0
Name: Varianza, Length: 38538, dtype: float64

In [896]:
dff.loc[dff["Varianza"]>250000]["Varianza"]

2        8.106157e+06
15       2.757937e+05
16       3.440489e+05
32       1.674450e+06
36       3.766563e+05
             ...     
40718    2.022060e+06
40720    8.632980e+05
40763    1.612808e+06
40784    7.482903e+05
40814    6.260805e+05
Name: Varianza, Length: 2381, dtype: float64

In [897]:
dff[:350]

,PurchDate,Model,SubModel,VehYear,Varianza
0,2009-01-04,1500 RAM,QUAD CAB,2002,0.0
1,2009-01-04,1500 RAM,QUAD CAB,2003,0.0
2,2009-01-04,1500 RAM,QUAD CAB,2004,8106157.0
3,2009-01-04,1500 RAM,QUAD CAB,2005,0.0
4,2009-01-04,300,SEDAN,2006,0.0
...,...,...,...,...,...
345,2009-01-09,ION,SEDAN,2005,0.0
346,2009-01-09,ION,SEDAN,2006,0.0
347,2009-01-09,LIBERTY,SUV,2003,0.0
348,2009-01-09,LIBERTY,SUV,2006,0.0


## Esempio di macchina con varianza alta
Controlliamo la stessa macchina vista in precedenza

In [898]:
dftest = df.loc[df['Model'] == '1500 RAM']

In [899]:
dftest = dftest.loc[dftest['SubModel'] == 'QUAD CAB']

In [900]:
dftest = dftest.loc[dftest['VehYear'] == 2004]

In [901]:
dftest = dftest.loc[dftest['PurchDate'] == '2009-01-04']

In [902]:
dftest[Variable]

7963     10890.0
22899     9676.0
57704     5465.0
Name: MMRAcquisitionRetailAveragePrice, dtype: float64

In [903]:
pd.options.display.max_columns = 40

In [904]:
dftest

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Model,Wheeldrive,Doors,SubModel
7963,46450,0,2009-01-04,MANHEIM,2004,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,81296,AMERICAN,LARGE TRUCK,CHRYSLER,9620.0,14210.0,10890.0,15847.0,9620.0,14210.0,10890.0,15847.0,NaN,NaN,19638,33809,FL,9460.0,0,1893,1500 RAM,4WD,NaN,QUAD CAB
22899,14101,0,2009-01-04,MANHEIM,2004,5,DODGE,5.7L,ST,NaN,BLACK,AUTO,3.0,Special,75251,AMERICAN,LARGE TRUCK,CHRYSLER,8496.0,10471.0,9676.0,11809.0,8496.0,10471.0,9676.0,11809.0,NaN,NaN,21053,92807,CA,9275.0,0,920,1500 RAM,2WD,NaN,QUAD CAB
57704,20577,0,2009-01-04,MANHEIM,2004,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,76276,AMERICAN,LARGE TRUCK,CHRYSLER,4597.0,6357.0,5465.0,7366.0,4597.0,6357.0,5465.0,7366.0,NaN,NaN,20392,75236,TX,9450.0,0,920,1500 RAM,2WD,NaN,QUAD CAB


fine test

In [905]:
mergedDF = pd.merge(dff, df, on=['PurchDate','Model', 'SubModel', 'VehYear'],  how='left')

In [906]:
mergedDF

,PurchDate,Model,SubModel,VehYear,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,2009-01-04,1500 RAM,QUAD CAB,2002,0.0,62472,0,ADESA,7,DODGE,5.9L,ST,NaN,BLUE,AUTO,1.0,Alloy,62209,AMERICAN,LARGE TRUCK,CHRYSLER,5872.0,7407.0,6842.0,8500.0,5872.0,7407.0,6842.0,8500.0,NaN,NaN,20234,85226,AZ,7940.0,0,1125,2WD,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,0.0,16342,0,MANHEIM,6,DODGE,5.7L,LAR,NaN,SILVER,AUTO,1.0,Alloy,79925,AMERICAN,LARGE TRUCK,CHRYSLER,7648.0,9717.0,8760.0,10994.0,7648.0,9717.0,8760.0,10994.0,NaN,NaN,19619,32824,FL,9400.0,0,1220,2WD,NaN
2,2009-01-04,1500 RAM,QUAD CAB,2004,8106157.0,46450,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,81296,AMERICAN,LARGE TRUCK,CHRYSLER,9620.0,14210.0,10890.0,15847.0,9620.0,14210.0,10890.0,15847.0,NaN,NaN,19638,33809,FL,9460.0,0,1893,4WD,NaN
3,2009-01-04,1500 RAM,QUAD CAB,2004,8106157.0,14101,0,MANHEIM,5,DODGE,5.7L,ST,NaN,BLACK,AUTO,3.0,Special,75251,AMERICAN,LARGE TRUCK,CHRYSLER,8496.0,10471.0,9676.0,11809.0,8496.0,10471.0,9676.0,11809.0,NaN,NaN,21053,92807,CA,9275.0,0,920,2WD,NaN
4,2009-01-04,1500 RAM,QUAD CAB,2004,8106157.0,20577,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,76276,AMERICAN,LARGE TRUCK,CHRYSLER,4597.0,6357.0,5465.0,7366.0,4597.0,6357.0,5465.0,7366.0,NaN,NaN,20392,75236,TX,9450.0,0,920,2WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,0.0,68096,1,ADESA,4,SUZUKI,NaN,BAS,NaN,BLACK,AUTO,1.0,NaN,55109,NOT AMERICAN,MEDIUM SUV,OTHER,7057.0,7950.0,10843.0,11604.0,7050.0,8061.0,11114.0,11830.0,NaN,NaN,52598,28273,NC,7470.0,0,920,2WD,4D
58382,2010-12-30,MALIBU,SEDAN,2005,0.0,60722,1,MANHEIM,5,CHEVROLET,4C,CLA,CLASSIC,SILVER,AUTO,2.0,Covers,80072,AMERICAN,MEDIUM,GM,3806.0,5397.0,6708.0,8588.0,3643.0,5100.0,6908.0,8446.0,NaN,NaN,21047,92337,CA,4395.0,0,1243,NaN,4D
58383,2010-12-30,SPECTRA,SEDAN,2006,0.0,70450,0,ADESA,4,KIA,NaN,EX,NaN,SILVER,AUTO,2.0,Covers,43116,NOT AMERICAN,MEDIUM,OTHER,4894.0,6120.0,8136.0,9384.0,4384.0,5253.0,7160.0,8535.0,NO,GREEN,20740,32219,FL,5350.0,0,482,NaN,4D
58384,2010-12-30,TAURUS,SEDAN,2006,0.0,68098,0,ADESA,4,FORD,NaN,SE,NaN,GREY,AUTO,2.0,Covers,69853,AMERICAN,MEDIUM,FORD,3964.0,5281.0,7182.0,9569.0,4683.0,5973.0,7881.0,9271.0,NO,GREEN,52598,28273,NC,5145.0,0,825,NaN,4D


In [907]:
dff2 = df[Variable].groupby([df['PurchDate'], df['Model'], df['SubModel'], df['VehYear']]).size().reset_index()

In [908]:
dff2.rename(columns={Variable:'Count'}, inplace=True)
dff2

,PurchDate,Model,SubModel,VehYear,Count
0,2009-01-04,1500 RAM,QUAD CAB,2002,1
1,2009-01-04,1500 RAM,QUAD CAB,2003,1
2,2009-01-04,1500 RAM,QUAD CAB,2004,3
3,2009-01-04,1500 RAM,QUAD CAB,2005,1
4,2009-01-04,300,SEDAN,2006,3
...,...,...,...,...,...
40914,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,1
40915,2010-12-30,MALIBU,SEDAN,2005,1
40916,2010-12-30,SPECTRA,SEDAN,2006,1
40917,2010-12-30,TAURUS,SEDAN,2006,1


In [909]:
mergedDF = pd.merge(dff2, mergedDF, on=['PurchDate','Model', 'SubModel', 'VehYear'],  how='left')

In [910]:
mergedDF

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,2009-01-04,1500 RAM,QUAD CAB,2002,1,0.0,62472,0,ADESA,7,DODGE,5.9L,ST,NaN,BLUE,AUTO,1.0,Alloy,62209,AMERICAN,LARGE TRUCK,CHRYSLER,5872.0,7407.0,6842.0,8500.0,5872.0,7407.0,6842.0,8500.0,NaN,NaN,20234,85226,AZ,7940.0,0,1125,2WD,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,1,0.0,16342,0,MANHEIM,6,DODGE,5.7L,LAR,NaN,SILVER,AUTO,1.0,Alloy,79925,AMERICAN,LARGE TRUCK,CHRYSLER,7648.0,9717.0,8760.0,10994.0,7648.0,9717.0,8760.0,10994.0,NaN,NaN,19619,32824,FL,9400.0,0,1220,2WD,NaN
2,2009-01-04,1500 RAM,QUAD CAB,2004,3,8106157.0,46450,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,81296,AMERICAN,LARGE TRUCK,CHRYSLER,9620.0,14210.0,10890.0,15847.0,9620.0,14210.0,10890.0,15847.0,NaN,NaN,19638,33809,FL,9460.0,0,1893,4WD,NaN
3,2009-01-04,1500 RAM,QUAD CAB,2004,3,8106157.0,14101,0,MANHEIM,5,DODGE,5.7L,ST,NaN,BLACK,AUTO,3.0,Special,75251,AMERICAN,LARGE TRUCK,CHRYSLER,8496.0,10471.0,9676.0,11809.0,8496.0,10471.0,9676.0,11809.0,NaN,NaN,21053,92807,CA,9275.0,0,920,2WD,NaN
4,2009-01-04,1500 RAM,QUAD CAB,2004,3,8106157.0,20577,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,76276,AMERICAN,LARGE TRUCK,CHRYSLER,4597.0,6357.0,5465.0,7366.0,4597.0,6357.0,5465.0,7366.0,NaN,NaN,20392,75236,TX,9450.0,0,920,2WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,1,0.0,68096,1,ADESA,4,SUZUKI,NaN,BAS,NaN,BLACK,AUTO,1.0,NaN,55109,NOT AMERICAN,MEDIUM SUV,OTHER,7057.0,7950.0,10843.0,11604.0,7050.0,8061.0,11114.0,11830.0,NaN,NaN,52598,28273,NC,7470.0,0,920,2WD,4D
58382,2010-12-30,MALIBU,SEDAN,2005,1,0.0,60722,1,MANHEIM,5,CHEVROLET,4C,CLA,CLASSIC,SILVER,AUTO,2.0,Covers,80072,AMERICAN,MEDIUM,GM,3806.0,5397.0,6708.0,8588.0,3643.0,5100.0,6908.0,8446.0,NaN,NaN,21047,92337,CA,4395.0,0,1243,NaN,4D
58383,2010-12-30,SPECTRA,SEDAN,2006,1,0.0,70450,0,ADESA,4,KIA,NaN,EX,NaN,SILVER,AUTO,2.0,Covers,43116,NOT AMERICAN,MEDIUM,OTHER,4894.0,6120.0,8136.0,9384.0,4384.0,5253.0,7160.0,8535.0,NO,GREEN,20740,32219,FL,5350.0,0,482,NaN,4D
58384,2010-12-30,TAURUS,SEDAN,2006,1,0.0,68098,0,ADESA,4,FORD,NaN,SE,NaN,GREY,AUTO,2.0,Covers,69853,AMERICAN,MEDIUM,FORD,3964.0,5281.0,7182.0,9569.0,4683.0,5973.0,7881.0,9271.0,NO,GREEN,52598,28273,NC,5145.0,0,825,NaN,4D


In [911]:
nulli= mergedDF.loc[mergedDF['Varianza']<1]

In [912]:
nulli

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,2009-01-04,1500 RAM,QUAD CAB,2002,1,0.0,62472,0,ADESA,7,DODGE,5.9L,ST,NaN,BLUE,AUTO,1.0,Alloy,62209,AMERICAN,LARGE TRUCK,CHRYSLER,5872.0,7407.0,6842.0,8500.0,5872.0,7407.0,6842.0,8500.0,NaN,NaN,20234,85226,AZ,7940.0,0,1125,2WD,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,1,0.0,16342,0,MANHEIM,6,DODGE,5.7L,LAR,NaN,SILVER,AUTO,1.0,Alloy,79925,AMERICAN,LARGE TRUCK,CHRYSLER,7648.0,9717.0,8760.0,10994.0,7648.0,9717.0,8760.0,10994.0,NaN,NaN,19619,32824,FL,9400.0,0,1220,2WD,NaN
5,2009-01-04,1500 RAM,QUAD CAB,2005,1,0.0,16344,0,MANHEIM,4,DODGE,5.7L,SLT,NaN,SILVER,AUTO,2.0,Covers,78806,AMERICAN,LARGE TRUCK,CHRYSLER,10306.0,12558.0,11630.0,14063.0,10306.0,12558.0,11630.0,14063.0,NaN,NaN,19619,32824,FL,10000.0,0,920,2WD,NaN
6,2009-01-04,300,SEDAN,2006,3,0.0,62463,0,ADESA,3,CHRYSLER,2.7L MPI,BAS,NaN,GREY,AUTO,2.0,Covers,71624,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20234,85226,AZ,9505.0,0,1389,NaN,4D
7,2009-01-04,300,SEDAN,2006,3,0.0,20574,0,MANHEIM,3,CHRYSLER,2.7L MPI,BAS,NaN,BLUE,AUTO,1.0,NaN,64278,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20833,75236,TX,10050.0,0,1215,NaN,4D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,1,0.0,68096,1,ADESA,4,SUZUKI,NaN,BAS,NaN,BLACK,AUTO,1.0,NaN,55109,NOT AMERICAN,MEDIUM SUV,OTHER,7057.0,7950.0,10843.0,11604.0,7050.0,8061.0,11114.0,11830.0,NaN,NaN,52598,28273,NC,7470.0,0,920,2WD,4D
58382,2010-12-30,MALIBU,SEDAN,2005,1,0.0,60722,1,MANHEIM,5,CHEVROLET,4C,CLA,CLASSIC,SILVER,AUTO,2.0,Covers,80072,AMERICAN,MEDIUM,GM,3806.0,5397.0,6708.0,8588.0,3643.0,5100.0,6908.0,8446.0,NaN,NaN,21047,92337,CA,4395.0,0,1243,NaN,4D
58383,2010-12-30,SPECTRA,SEDAN,2006,1,0.0,70450,0,ADESA,4,KIA,NaN,EX,NaN,SILVER,AUTO,2.0,Covers,43116,NOT AMERICAN,MEDIUM,OTHER,4894.0,6120.0,8136.0,9384.0,4384.0,5253.0,7160.0,8535.0,NO,GREEN,20740,32219,FL,5350.0,0,482,NaN,4D
58384,2010-12-30,TAURUS,SEDAN,2006,1,0.0,68098,0,ADESA,4,FORD,NaN,SE,NaN,GREY,AUTO,2.0,Covers,69853,AMERICAN,MEDIUM,FORD,3964.0,5281.0,7182.0,9569.0,4683.0,5973.0,7881.0,9271.0,NO,GREEN,52598,28273,NC,5145.0,0,825,NaN,4D


In [913]:
nulli= mergedDF.loc[mergedDF['Varianza']<=250000]

In [914]:
nulli

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,2009-01-04,1500 RAM,QUAD CAB,2002,1,0.0,62472,0,ADESA,7,DODGE,5.9L,ST,NaN,BLUE,AUTO,1.0,Alloy,62209,AMERICAN,LARGE TRUCK,CHRYSLER,5872.0,7407.0,6842.0,8500.0,5872.0,7407.0,6842.0,8500.0,NaN,NaN,20234,85226,AZ,7940.0,0,1125,2WD,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,1,0.0,16342,0,MANHEIM,6,DODGE,5.7L,LAR,NaN,SILVER,AUTO,1.0,Alloy,79925,AMERICAN,LARGE TRUCK,CHRYSLER,7648.0,9717.0,8760.0,10994.0,7648.0,9717.0,8760.0,10994.0,NaN,NaN,19619,32824,FL,9400.0,0,1220,2WD,NaN
5,2009-01-04,1500 RAM,QUAD CAB,2005,1,0.0,16344,0,MANHEIM,4,DODGE,5.7L,SLT,NaN,SILVER,AUTO,2.0,Covers,78806,AMERICAN,LARGE TRUCK,CHRYSLER,10306.0,12558.0,11630.0,14063.0,10306.0,12558.0,11630.0,14063.0,NaN,NaN,19619,32824,FL,10000.0,0,920,2WD,NaN
6,2009-01-04,300,SEDAN,2006,3,0.0,62463,0,ADESA,3,CHRYSLER,2.7L MPI,BAS,NaN,GREY,AUTO,2.0,Covers,71624,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20234,85226,AZ,9505.0,0,1389,NaN,4D
7,2009-01-04,300,SEDAN,2006,3,0.0,20574,0,MANHEIM,3,CHRYSLER,2.7L MPI,BAS,NaN,BLUE,AUTO,1.0,NaN,64278,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20833,75236,TX,10050.0,0,1215,NaN,4D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,1,0.0,68096,1,ADESA,4,SUZUKI,NaN,BAS,NaN,BLACK,AUTO,1.0,NaN,55109,NOT AMERICAN,MEDIUM SUV,OTHER,7057.0,7950.0,10843.0,11604.0,7050.0,8061.0,11114.0,11830.0,NaN,NaN,52598,28273,NC,7470.0,0,920,2WD,4D
58382,2010-12-30,MALIBU,SEDAN,2005,1,0.0,60722,1,MANHEIM,5,CHEVROLET,4C,CLA,CLASSIC,SILVER,AUTO,2.0,Covers,80072,AMERICAN,MEDIUM,GM,3806.0,5397.0,6708.0,8588.0,3643.0,5100.0,6908.0,8446.0,NaN,NaN,21047,92337,CA,4395.0,0,1243,NaN,4D
58383,2010-12-30,SPECTRA,SEDAN,2006,1,0.0,70450,0,ADESA,4,KIA,NaN,EX,NaN,SILVER,AUTO,2.0,Covers,43116,NOT AMERICAN,MEDIUM,OTHER,4894.0,6120.0,8136.0,9384.0,4384.0,5253.0,7160.0,8535.0,NO,GREEN,20740,32219,FL,5350.0,0,482,NaN,4D
58384,2010-12-30,TAURUS,SEDAN,2006,1,0.0,68098,0,ADESA,4,FORD,NaN,SE,NaN,GREY,AUTO,2.0,Covers,69853,AMERICAN,MEDIUM,FORD,3964.0,5281.0,7182.0,9569.0,4683.0,5973.0,7881.0,9271.0,NO,GREEN,52598,28273,NC,5145.0,0,825,NaN,4D


In [915]:
nulli= mergedDF.loc[mergedDF['Varianza']>250000]

## 6433 macchine non concordano con la definizione di MMR

In [916]:
nulli

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
2,2009-01-04,1500 RAM,QUAD CAB,2004,3,8.106157e+06,46450,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,81296,AMERICAN,LARGE TRUCK,CHRYSLER,9620.0,14210.0,10890.0,15847.0,9620.0,14210.0,10890.0,15847.0,NaN,NaN,19638,33809,FL,9460.0,0,1893,4WD,NaN
3,2009-01-04,1500 RAM,QUAD CAB,2004,3,8.106157e+06,14101,0,MANHEIM,5,DODGE,5.7L,ST,NaN,BLACK,AUTO,3.0,Special,75251,AMERICAN,LARGE TRUCK,CHRYSLER,8496.0,10471.0,9676.0,11809.0,8496.0,10471.0,9676.0,11809.0,NaN,NaN,21053,92807,CA,9275.0,0,920,2WD,NaN
4,2009-01-04,1500 RAM,QUAD CAB,2004,3,8.106157e+06,20577,0,MANHEIM,5,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,76276,AMERICAN,LARGE TRUCK,CHRYSLER,4597.0,6357.0,5465.0,7366.0,4597.0,6357.0,5465.0,7366.0,NaN,NaN,20392,75236,TX,9450.0,0,920,2WD,NaN
19,2009-01-04,CARAVAN,MINIVAN,2003,5,2.757937e+05,16363,0,MANHEIM,6,DODGE,4C 2.4L,SE,NaN,BLUE,AUTO,2.0,Covers,76263,AMERICAN,VAN,CHRYSLER,2606.0,3423.0,3314.0,4197.0,2606.0,3423.0,3314.0,4197.0,NaN,NaN,20928,32824,FL,5000.0,0,1923,2WD,NaN
20,2009-01-04,CARAVAN,MINIVAN,2003,5,2.757937e+05,42857,0,MANHEIM,6,DODGE,3.8L,SPO,SPORT,RED,AUTO,1.0,Alloy,56930,AMERICAN,VAN,CHRYSLER,3903.0,4790.0,4715.0,5673.0,3903.0,4790.0,4715.0,5673.0,NaN,NaN,21973,32124,FL,5700.0,0,1551,2WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58233,2010-12-22,IMPALA,SEDAN,2005,3,7.482903e+05,2980,1,OTHER,5,CHEVROLET,NaN,LS,NaN,WHITE,AUTO,1.0,NaN,108330,AMERICAN,LARGE,GM,4947.0,6287.0,8295.0,10038.0,4795.0,6113.0,7968.0,9287.0,NaN,NaN,18880,35004,AL,4100.0,0,2571,NaN,4D
58234,2010-12-22,IMPALA,SEDAN,2005,3,7.482903e+05,23757,0,MANHEIM,5,CHEVROLET,NaN,BAS,NaN,GOLD,AUTO,2.0,Covers,84367,AMERICAN,LARGE,GM,3782.0,5024.0,6780.0,8395.0,4013.0,5328.0,6674.0,8360.0,NO,GREEN,23359,92807,CA,5330.0,0,2152,NaN,4D
58235,2010-12-22,IMPALA,SEDAN,2005,3,7.482903e+05,10273,0,MANHEIM,5,CHEVROLET,NaN,BAS,NaN,SILVER,AUTO,2.0,Covers,93386,AMERICAN,LARGE,GM,3808.0,5058.0,6814.0,8439.0,4032.0,5391.0,7072.0,8797.0,NO,GREEN,21047,94544,CA,5220.0,0,2351,NaN,4D
58275,2010-12-22,SABLE,SEDAN,2002,2,6.260805e+05,52397,0,MANHEIM,8,MERCURY,NaN,GS,NaN,GOLD,AUTO,2.0,Covers,109662,AMERICAN,MEDIUM,FORD,2183.0,3047.0,4244.0,5604.0,2204.0,3214.0,4973.0,5740.0,NO,GREEN,53245,78219,TX,4235.0,0,1430,NaN,4D


In [917]:
varbassa = mergedDF.loc[mergedDF['Varianza']<250000]

In [918]:
varbassa

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,2009-01-04,1500 RAM,QUAD CAB,2002,1,0.0,62472,0,ADESA,7,DODGE,5.9L,ST,NaN,BLUE,AUTO,1.0,Alloy,62209,AMERICAN,LARGE TRUCK,CHRYSLER,5872.0,7407.0,6842.0,8500.0,5872.0,7407.0,6842.0,8500.0,NaN,NaN,20234,85226,AZ,7940.0,0,1125,2WD,NaN
1,2009-01-04,1500 RAM,QUAD CAB,2003,1,0.0,16342,0,MANHEIM,6,DODGE,5.7L,LAR,NaN,SILVER,AUTO,1.0,Alloy,79925,AMERICAN,LARGE TRUCK,CHRYSLER,7648.0,9717.0,8760.0,10994.0,7648.0,9717.0,8760.0,10994.0,NaN,NaN,19619,32824,FL,9400.0,0,1220,2WD,NaN
5,2009-01-04,1500 RAM,QUAD CAB,2005,1,0.0,16344,0,MANHEIM,4,DODGE,5.7L,SLT,NaN,SILVER,AUTO,2.0,Covers,78806,AMERICAN,LARGE TRUCK,CHRYSLER,10306.0,12558.0,11630.0,14063.0,10306.0,12558.0,11630.0,14063.0,NaN,NaN,19619,32824,FL,10000.0,0,920,2WD,NaN
6,2009-01-04,300,SEDAN,2006,3,0.0,62463,0,ADESA,3,CHRYSLER,2.7L MPI,BAS,NaN,GREY,AUTO,2.0,Covers,71624,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20234,85226,AZ,9505.0,0,1389,NaN,4D
7,2009-01-04,300,SEDAN,2006,3,0.0,20574,0,MANHEIM,3,CHRYSLER,2.7L MPI,BAS,NaN,BLUE,AUTO,1.0,NaN,64278,AMERICAN,SPECIALTY,CHRYSLER,9459.0,11179.0,10716.0,12573.0,9459.0,11179.0,10716.0,12573.0,NaN,NaN,20833,75236,TX,10050.0,0,1215,NaN,4D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,2010-12-30,GRAND VITARA,UTILITY SPORT,2006,1,0.0,68096,1,ADESA,4,SUZUKI,NaN,BAS,NaN,BLACK,AUTO,1.0,NaN,55109,NOT AMERICAN,MEDIUM SUV,OTHER,7057.0,7950.0,10843.0,11604.0,7050.0,8061.0,11114.0,11830.0,NaN,NaN,52598,28273,NC,7470.0,0,920,2WD,4D
58382,2010-12-30,MALIBU,SEDAN,2005,1,0.0,60722,1,MANHEIM,5,CHEVROLET,4C,CLA,CLASSIC,SILVER,AUTO,2.0,Covers,80072,AMERICAN,MEDIUM,GM,3806.0,5397.0,6708.0,8588.0,3643.0,5100.0,6908.0,8446.0,NaN,NaN,21047,92337,CA,4395.0,0,1243,NaN,4D
58383,2010-12-30,SPECTRA,SEDAN,2006,1,0.0,70450,0,ADESA,4,KIA,NaN,EX,NaN,SILVER,AUTO,2.0,Covers,43116,NOT AMERICAN,MEDIUM,OTHER,4894.0,6120.0,8136.0,9384.0,4384.0,5253.0,7160.0,8535.0,NO,GREEN,20740,32219,FL,5350.0,0,482,NaN,4D
58384,2010-12-30,TAURUS,SEDAN,2006,1,0.0,68098,0,ADESA,4,FORD,NaN,SE,NaN,GREY,AUTO,2.0,Covers,69853,AMERICAN,MEDIUM,FORD,3964.0,5281.0,7182.0,9569.0,4683.0,5973.0,7881.0,9271.0,NO,GREEN,52598,28273,NC,5145.0,0,825,NaN,4D


In [919]:
contvarbassa = varbassa.loc[varbassa['Count']>5]

In [920]:
contvarbassa

,PurchDate,Model,SubModel,VehYear,Count,Varianza,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
177,2009-01-05,AVENGER,SEDAN,2008,12,0.0,55791,0,OTHER,1,DODGE,4C 2.4L,SE,NaN,BLACK,AUTO,2.0,Covers,38212,AMERICAN,MEDIUM,CHRYSLER,8196.0,9044.0,9352.0,10268.0,8196.0,9044.0,9352.0,10268.0,NaN,NaN,99761,33073,FL,8200.0,0,762,NaN,4D
178,2009-01-05,AVENGER,SEDAN,2008,12,0.0,55795,0,OTHER,1,DODGE,4C 2.4L,SE,NaN,BLACK,AUTO,2.0,Covers,38085,AMERICAN,MEDIUM,CHRYSLER,8196.0,9044.0,9352.0,10268.0,8196.0,9044.0,9352.0,10268.0,NaN,NaN,99761,33073,FL,8200.0,0,762,NaN,4D
179,2009-01-05,AVENGER,SEDAN,2008,12,0.0,55773,0,OTHER,1,DODGE,4C 2.4L,SE,NaN,BLACK,AUTO,2.0,Covers,38978,AMERICAN,MEDIUM,CHRYSLER,8196.0,9044.0,9352.0,10268.0,8196.0,9044.0,9352.0,10268.0,NaN,NaN,99761,33073,FL,8200.0,0,762,NaN,4D
180,2009-01-05,AVENGER,SEDAN,2008,12,0.0,55789,0,OTHER,1,DODGE,4C 2.4L,SE,NaN,BLUE,AUTO,2.0,Covers,39066,AMERICAN,MEDIUM,CHRYSLER,8196.0,9044.0,9352.0,10268.0,8196.0,9044.0,9352.0,10268.0,NaN,NaN,99761,33073,FL,8200.0,0,762,NaN,4D
181,2009-01-05,AVENGER,SEDAN,2008,12,0.0,55772,0,OTHER,1,DODGE,4C 2.4L,SE,NaN,WHITE,AUTO,2.0,Covers,47529,AMERICAN,MEDIUM,CHRYSLER,8196.0,9044.0,9352.0,10268.0,8196.0,9044.0,9352.0,10268.0,NaN,NaN,99761,33073,FL,8200.0,0,834,NaN,4D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57697,2010-12-15,IMPALA,SEDAN,2007,6,2774.4,44610,1,MANHEIM,3,CHEVROLET,NaN,LS,NaN,GOLD,AUTO,2.0,Covers,69306,AMERICAN,LARGE,GM,7345.0,8489.0,10022.0,11498.0,7078.0,8282.0,10791.0,12229.0,NaN,NaN,99750,32124,FL,8400.0,0,1703,NaN,4D
57698,2010-12-15,IMPALA,SEDAN,2007,6,2774.4,54981,0,OTHER,3,CHEVROLET,NaN,LS,NaN,MAROON,AUTO,2.0,Covers,91345,AMERICAN,LARGE,GM,7424.0,8595.0,10124.0,11634.0,7318.0,8582.0,11169.0,12449.0,NaN,NaN,22808,73108,OK,7300.0,0,2351,NaN,4D
57699,2010-12-15,IMPALA,SEDAN,2007,6,2774.4,44606,0,MANHEIM,3,CHEVROLET,NaN,LS,NaN,GOLD,AUTO,2.0,Covers,66794,AMERICAN,LARGE,GM,7345.0,8489.0,10022.0,11498.0,6814.0,7885.0,10481.0,11725.0,NaN,NaN,99750,32124,FL,8200.0,0,1703,NaN,4D
57700,2010-12-15,IMPALA,SEDAN,2007,6,2774.4,54976,0,OTHER,3,CHEVROLET,NaN,LS,NaN,SILVER,AUTO,1.0,Alloy,73369,AMERICAN,LARGE,GM,7424.0,8595.0,10124.0,11634.0,7757.0,9024.0,11137.0,12326.0,NO,GREEN,22808,73108,OK,7300.0,0,1974,NaN,4D


In [921]:
varbassa['Count'].max()

63

In [922]:
dffsenzadata

,Model,SubModel,VehYear,MMRAcquisitionRetailAveragePrice
0,1500 RAM,EXT CAB,2001,1.984500e+05
1,1500 RAM,MEGA CAB,2006,3.280172e+06
2,1500 RAM,QUAD CAB,2002,4.577251e+06
3,1500 RAM,QUAD CAB,2003,5.089070e+06
4,1500 RAM,QUAD CAB,2004,6.954771e+06
...,...,...,...,...
1383,YUKON XL,UTILITY,2002,2.250199e+06
1384,YUKON XL,UTILITY,2003,2.675061e+06
1385,YUKON XL,UTILITY,2004,2.523528e+06
1386,YUKON XL,UTILITY,2005,NaN


## Analisi dffsenzadata

In [923]:
dffsenzadata.rename(columns={Variable:'Varianza'}, inplace=True)
dffsenzadata

,Model,SubModel,VehYear,Varianza
0,1500 RAM,EXT CAB,2001,1.984500e+05
1,1500 RAM,MEGA CAB,2006,3.280172e+06
2,1500 RAM,QUAD CAB,2002,4.577251e+06
3,1500 RAM,QUAD CAB,2003,5.089070e+06
4,1500 RAM,QUAD CAB,2004,6.954771e+06
...,...,...,...,...
1383,YUKON XL,UTILITY,2002,2.250199e+06
1384,YUKON XL,UTILITY,2003,2.675061e+06
1385,YUKON XL,UTILITY,2004,2.523528e+06
1386,YUKON XL,UTILITY,2005,NaN


In [924]:
dffsenzadata['Varianza'].isnull().sum()

256

In [925]:
dffsenzadata['Varianza'] = dffsenzadata['Varianza'].fillna(0)

## Varianza dffsenzadata
all'incirca il 1.5% del database ha una varianza inferiore alla soglia scelta (250000) mentre il 0.4% ha una varianza pari a 0

In [926]:
mergedDF2 = pd.merge(dffsenzadata, df, on=['Model', 'SubModel', 'VehYear'],  how='left')

In [927]:
mergedDF2

,Model,SubModel,VehYear,Varianza,RefId,IsBadBuy,PurchDate,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,1500 RAM,EXT CAB,2001,1.984500e+05,68125,0,2009-01-29,ADESA,8,DODGE,5.9L,LAR,LARAMIE,BLACK,AUTO,1.0,Alloy,87847,AMERICAN,LARGE TRUCK,CHRYSLER,2608.0,3516.0,3317.0,4297.0,3392.0,4436.0,4163.0,5291.0,NaN,NaN,3453,80817,CO,4480.0,0,1283,2WD,4D
1,1500 RAM,EXT CAB,2001,1.984500e+05,61608,1,2009-07-28,ADESA,8,DODGE,5.2L,LAR,LARAMIE,BLUE,AUTO,1.0,NaN,55117,AMERICAN,LARGE TRUCK,CHRYSLER,3192.0,4269.0,3947.0,5111.0,3192.0,4269.0,3947.0,5111.0,NaN,NaN,23359,91752,CA,4315.0,0,1054,2WD,4D
2,1500 RAM,MEGA CAB,2006,3.280172e+06,50261,0,2009-02-25,MANHEIM,3,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,72728,AMERICAN,LARGE TRUCK,CHRYSLER,12423.0,15017.0,13917.0,16718.0,13721.0,15229.0,15319.0,16947.0,NaN,NaN,19662,78219,TX,10835.0,0,920,2WD,NaN
3,1500 RAM,MEGA CAB,2006,3.280172e+06,33703,0,2009-01-12,MANHEIM,3,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,80650,AMERICAN,LARGE TRUCK,CHRYSLER,13271.0,15154.0,14833.0,16866.0,12982.0,14842.0,18542.0,20697.0,NaN,NaN,19619,34761,FL,11515.0,0,983,2WD,NaN
4,1500 RAM,MEGA CAB,2006,3.280172e+06,39145,0,2010-03-18,MANHEIM,4,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,91710,AMERICAN,LARGE TRUCK,CHRYSLER,13227.0,16033.0,17410.0,19843.0,14571.0,15835.0,18891.0,20220.0,NaN,NaN,17212,76040,TX,10550.0,0,1053,2WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,ZEPHYR,SEDAN,2006,6.381721e+05,70031,0,2010-07-14,ADESA,4,LINCOLN,NaN,BAS,NaN,RED,AUTO,1.0,Alloy,70797,AMERICAN,SPECIALTY,FORD,12532.0,14479.0,16113.0,17482.0,11528.0,14189.0,15602.0,18140.0,NaN,NaN,21053,85226,AZ,8615.0,0,1272,NaN,4D
58382,ZEPHYR,SEDAN,2006,6.381721e+05,25215,0,2010-02-18,OTHER,4,LINCOLN,NaN,BAS,NaN,GOLD,AUTO,1.0,Alloy,82286,AMERICAN,SPECIALTY,FORD,12341.0,13964.0,16405.0,17844.0,13115.0,14984.0,16693.0,18439.0,NaN,NaN,3453,80229,CO,8675.0,0,1373,NaN,4D
58383,ZEPHYR,SEDAN,2006,6.381721e+05,32556,0,2009-03-03,MANHEIM,3,LINCOLN,NaN,BAS,NaN,BEIGE,AUTO,1.0,Alloy,68976,AMERICAN,SPECIALTY,FORD,13765.0,15297.0,15366.0,17021.0,13981.0,15490.0,15599.0,17229.0,NaN,NaN,3453,80022,CO,9215.0,0,1118,NaN,4D
58384,ZEPHYR,SEDAN,2006,6.381721e+05,13079,0,2010-10-18,MANHEIM,4,LINCOLN,NaN,BAS,NaN,BEIGE,AUTO,1.0,Alloy,76988,AMERICAN,SPECIALTY,FORD,10448.0,12558.0,14819.0,17078.0,10098.0,12489.0,14519.0,16362.0,NO,GREEN,17675,27542,NC,10000.0,0,1272,NaN,4D


In [928]:
nulli= mergedDF2.loc[mergedDF2['Varianza']<1]

In [929]:
nulli

,Model,SubModel,VehYear,Varianza,RefId,IsBadBuy,PurchDate,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
926,1500 RAM,REG CAB,2008,0.0,37121,0,2010-06-30,MANHEIM,2,DODGE,3.7L,ST,NaN,RED,AUTO,2.0,Covers,67650,AMERICAN,LARGE TRUCK,CHRYSLER,10001.0,12108.0,15341.0,16741.0,8252.0,8661.0,14080.0,14307.0,NaN,NaN,25100,75236,TX,7940.0,0,533,2WD,NaN
927,1500 SIERRA,CREW CAB,2004,0.0,48253,0,2009-10-02,OTHER,5,GMC,5.3L,SLE,NaN,BEIGE,AUTO,1.0,Alloy,87442,AMERICAN,LARGE TRUCK,GM,8541.0,10491.0,9724.0,11830.0,9160.0,10126.0,10393.0,11436.0,NaN,NaN,19662,70401,LA,10500.0,0,1243,NaN,NaN
1525,1500HD SILVERADO,CREW CAB,2006,0.0,22472,0,2009-06-18,MANHEIM,3,CHEVROLET,6.0L,LT,NaN,WHITE,AUTO,1.0,Alloy,88928,AMERICAN,LARGE TRUCK,GM,12730.0,16220.0,14248.0,18018.0,10909.0,13903.0,12282.0,15515.0,NaN,NaN,17212,76040,TX,11790.0,0,1243,NaN,NaN
1535,2500 RAM,QUAD CAB,2006,0.0,49328,1,2010-10-06,OTHER,4,DODGE,5.7L,LAR,NaN,WHITE,AUTO,1.0,Alloy,86735,AMERICAN,LARGE TRUCK,CHRYSLER,10061.0,12975.0,15103.0,18153.0,8086.0,11222.0,12800.0,16319.0,NaN,NaN,20833,73108,OK,10955.0,0,983,2WD,NaN
1536,2500 RAM,QUAD CAB,2007,0.0,5821,0,2010-12-07,MANHEIM,3,DODGE,5.7L,LAR,BIG HORN,RED,AUTO,2.0,Covers,92488,AMERICAN,LARGE TRUCK,CHRYSLER,18171.0,22252.0,23613.0,27513.0,18320.0,20843.0,23451.0,26375.0,NaN,NaN,99750,85040,AZ,11000.0,0,2062,4WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58054,XL-7,UTILITY SPORT,2003,0.0,37819,0,2010-10-06,MANHEIM,7,SUZUKI,NaN,TOU,NaN,WHITE,AUTO,1.0,Alloy,58265,NOT AMERICAN,MEDIUM SUV,OTHER,5145.0,6267.0,8490.0,9629.0,5674.0,6809.0,9034.0,10306.0,NaN,NaN,22916,89120,NV,7620.0,0,1551,4WD,4D
58272,YARIS,HATCHBACK,2009,0.0,31196,0,2010-09-30,OTHER,1,TOYOTA,NaN,BAS,NaN,BLACK,AUTO,2.0,Covers,49368,NOT AMERICAN,COMPACT,OTHER,8251.0,8859.0,10577.0,11179.0,7885.0,8618.0,9981.0,10909.0,NaN,NaN,99761,85284,AZ,7550.0,0,482,NaN,2D
58273,YARIS,HATCHBACK,2009,0.0,31200,0,2010-09-30,OTHER,1,TOYOTA,NaN,BAS,NaN,WHITE,AUTO,2.0,Covers,47773,NOT AMERICAN,COMPACT,OTHER,8251.0,8859.0,10577.0,11179.0,8245.0,8767.0,10428.0,11089.0,NaN,NaN,99761,85284,AZ,7550.0,0,482,NaN,2D
58288,YARIS,SEDAN,2009,0.0,31155,0,2010-09-30,OTHER,1,TOYOTA,NaN,BAS,NaN,SILVER,AUTO,2.0,Covers,49459,NOT AMERICAN,COMPACT,OTHER,9054.0,9696.0,11372.0,12008.0,9173.0,10044.0,11329.0,12123.0,NaN,NaN,99761,85284,AZ,7550.0,0,482,NaN,4D


In [930]:
nulli= mergedDF2.loc[mergedDF2['Varianza']<=250000]

In [931]:
nulli

,Model,SubModel,VehYear,Varianza,RefId,IsBadBuy,PurchDate,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
0,1500 RAM,EXT CAB,2001,198450.000000,68125,0,2009-01-29,ADESA,8,DODGE,5.9L,LAR,LARAMIE,BLACK,AUTO,1.0,Alloy,87847,AMERICAN,LARGE TRUCK,CHRYSLER,2608.0,3516.0,3317.0,4297.0,3392.0,4436.0,4163.0,5291.0,NaN,NaN,3453,80817,CO,4480.0,0,1283,2WD,4D
1,1500 RAM,EXT CAB,2001,198450.000000,61608,1,2009-07-28,ADESA,8,DODGE,5.2L,LAR,LARAMIE,BLUE,AUTO,1.0,NaN,55117,AMERICAN,LARGE TRUCK,CHRYSLER,3192.0,4269.0,3947.0,5111.0,3192.0,4269.0,3947.0,5111.0,NaN,NaN,23359,91752,CA,4315.0,0,1054,2WD,4D
926,1500 RAM,REG CAB,2008,0.000000,37121,0,2010-06-30,MANHEIM,2,DODGE,3.7L,ST,NaN,RED,AUTO,2.0,Covers,67650,AMERICAN,LARGE TRUCK,CHRYSLER,10001.0,12108.0,15341.0,16741.0,8252.0,8661.0,14080.0,14307.0,NaN,NaN,25100,75236,TX,7940.0,0,533,2WD,NaN
927,1500 SIERRA,CREW CAB,2004,0.000000,48253,0,2009-10-02,OTHER,5,GMC,5.3L,SLE,NaN,BEIGE,AUTO,1.0,Alloy,87442,AMERICAN,LARGE TRUCK,GM,8541.0,10491.0,9724.0,11830.0,9160.0,10126.0,10393.0,11436.0,NaN,NaN,19662,70401,LA,10500.0,0,1243,NaN,NaN
1027,1500 SIERRA,REG CAB,2007,33812.333333,43262,0,2010-02-06,MANHEIM,3,GMC,4.3L,SL,NaN,WHITE,AUTO,2.0,Covers,66974,AMERICAN,LARGE TRUCK,GM,9015.0,11466.0,12624.0,16071.0,8798.0,10682.0,12264.0,14387.0,NaN,NaN,19619,32124,FL,6450.0,0,1118,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58280,YARIS,SEDAN,2007,235873.944444,10015,0,2010-07-12,MANHEIM,3,TOYOTA,NaN,BAS,NaN,GOLD,AUTO,2.0,Covers,70095,NOT AMERICAN,COMPACT,OTHER,6860.0,7884.0,9706.0,10652.0,6711.0,7754.0,10020.0,11421.0,NaN,NaN,52646,21075,MD,6485.0,0,569,NaN,4D
58281,YARIS,SEDAN,2007,235873.944444,33879,0,2010-06-04,MANHEIM,3,TOYOTA,NaN,BAS,NaN,GREEN,AUTO,3.0,Special,59532,NOT AMERICAN,COMPACT,OTHER,7172.0,7977.0,11196.0,12453.0,7237.0,8187.0,10429.0,11427.0,NaN,NaN,19638,34761,FL,7485.0,0,505,NaN,4D
58282,YARIS,SEDAN,2007,235873.944444,66737,0,2010-05-13,ADESA,3,TOYOTA,NaN,BAS,NaN,BLACK,AUTO,2.0,Covers,93179,NOT AMERICAN,COMPACT,OTHER,6710.0,7675.0,10086.0,10987.0,6750.0,7793.0,9989.0,11563.0,NaN,NaN,17675,28273,NC,6190.0,0,620,NaN,4D
58288,YARIS,SEDAN,2009,0.000000,31155,0,2010-09-30,OTHER,1,TOYOTA,NaN,BAS,NaN,SILVER,AUTO,2.0,Covers,49459,NOT AMERICAN,COMPACT,OTHER,9054.0,9696.0,11372.0,12008.0,9173.0,10044.0,11329.0,12123.0,NaN,NaN,99761,85284,AZ,7550.0,0,482,NaN,4D


In [932]:
nulli= mergedDF2.loc[mergedDF2['Varianza']>250000]

## 57429 macchine con rispettano la soglia

In [933]:
nulli

,Model,SubModel,VehYear,Varianza,RefId,IsBadBuy,PurchDate,Auction,VehicleAge,Make,Engine,Trim,SubModelSpecifics,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Wheeldrive,Doors
2,1500 RAM,MEGA CAB,2006,3.280172e+06,50261,0,2009-02-25,MANHEIM,3,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,72728,AMERICAN,LARGE TRUCK,CHRYSLER,12423.0,15017.0,13917.0,16718.0,13721.0,15229.0,15319.0,16947.0,NaN,NaN,19662,78219,TX,10835.0,0,920,2WD,NaN
3,1500 RAM,MEGA CAB,2006,3.280172e+06,33703,0,2009-01-12,MANHEIM,3,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,80650,AMERICAN,LARGE TRUCK,CHRYSLER,13271.0,15154.0,14833.0,16866.0,12982.0,14842.0,18542.0,20697.0,NaN,NaN,19619,34761,FL,11515.0,0,983,2WD,NaN
4,1500 RAM,MEGA CAB,2006,3.280172e+06,39145,0,2010-03-18,MANHEIM,4,DODGE,5.7L,LAR,NaN,RED,AUTO,1.0,Alloy,91710,AMERICAN,LARGE TRUCK,CHRYSLER,13227.0,16033.0,17410.0,19843.0,14571.0,15835.0,18891.0,20220.0,NaN,NaN,17212,76040,TX,10550.0,0,1053,2WD,NaN
5,1500 RAM,QUAD CAB,2002,4.577251e+06,63315,0,2010-11-30,MANHEIM,8,DODGE,4.7L,ST,NaN,GOLD,AUTO,1.0,Alloy,84794,AMERICAN,LARGE TRUCK,CHRYSLER,5890.0,7501.0,8971.0,10374.0,5171.0,6304.0,8729.0,9573.0,NO,GREEN,52598,28625,NC,9110.0,0,1283,2WD,NaN
6,1500 RAM,QUAD CAB,2002,4.577251e+06,36501,0,2009-02-10,MANHEIM,7,DODGE,4.7L,ST,NaN,BLUE,AUTO,1.0,Alloy,89552,AMERICAN,LARGE TRUCK,CHRYSLER,3559.0,3997.0,4344.0,4817.0,3942.0,5594.0,6841.0,9425.0,NaN,NaN,17212,75236,TX,7650.0,0,1283,2WD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,ZEPHYR,SEDAN,2006,6.381721e+05,70031,0,2010-07-14,ADESA,4,LINCOLN,NaN,BAS,NaN,RED,AUTO,1.0,Alloy,70797,AMERICAN,SPECIALTY,FORD,12532.0,14479.0,16113.0,17482.0,11528.0,14189.0,15602.0,18140.0,NaN,NaN,21053,85226,AZ,8615.0,0,1272,NaN,4D
58382,ZEPHYR,SEDAN,2006,6.381721e+05,25215,0,2010-02-18,OTHER,4,LINCOLN,NaN,BAS,NaN,GOLD,AUTO,1.0,Alloy,82286,AMERICAN,SPECIALTY,FORD,12341.0,13964.0,16405.0,17844.0,13115.0,14984.0,16693.0,18439.0,NaN,NaN,3453,80229,CO,8675.0,0,1373,NaN,4D
58383,ZEPHYR,SEDAN,2006,6.381721e+05,32556,0,2009-03-03,MANHEIM,3,LINCOLN,NaN,BAS,NaN,BEIGE,AUTO,1.0,Alloy,68976,AMERICAN,SPECIALTY,FORD,13765.0,15297.0,15366.0,17021.0,13981.0,15490.0,15599.0,17229.0,NaN,NaN,3453,80022,CO,9215.0,0,1118,NaN,4D
58384,ZEPHYR,SEDAN,2006,6.381721e+05,13079,0,2010-10-18,MANHEIM,4,LINCOLN,NaN,BAS,NaN,BEIGE,AUTO,1.0,Alloy,76988,AMERICAN,SPECIALTY,FORD,10448.0,12558.0,14819.0,17078.0,10098.0,12489.0,14519.0,16362.0,NO,GREEN,17675,27542,NC,10000.0,0,1272,NaN,4D


In [934]:
mergedDF2['Varianza'].describe()

count    5.838600e+04
mean     1.782009e+06
std      1.413993e+06
min      0.000000e+00
25%      1.166357e+06
50%      1.571481e+06
75%      2.016683e+06
max      1.476621e+08
Name: Varianza, dtype: float64

In [935]:
mergedDF['Varianza'].describe()

count    5.838600e+04
mean     1.625278e+05
std      7.892021e+05
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      8.702500e+02
max      3.590434e+07
Name: Varianza, dtype: float64